In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sts
from scipy import stats
from sklearn.linear_model import LinearRegression
import plotly.express as px
from scipy.stats import normaltest
import requests
from bs4 import BeautifulSoup
from scipy.stats import pearsonr

WEB Scraming

In [16]:
brands=[]
models=[]
prices=[]
favorite_number=[]
rush_delivery=[]
Free_cargo=[]
rating=[]
links=[]

In [30]:
with open("Computer - Trendyol.html", "r", encoding="utf-8") as file:
    html_content = file.read()


soup = BeautifulSoup(html_content, 'html.parser')

for pc in soup.find_all('div',class_="p-card-wrppr with-campaign-view"):
    brand_tag=pc.find("span",class_="prdct-desc-cntnr-ttl")
    brand = brand_tag.text.strip() if brand_tag else np.nan 
    brands.append(brand)
    
    model_tag = pc.find("span", class_="prdct-desc-cntnr-name hasRatings")
    model_tag1 = pc.find("div", class_="product-desc-sub-text")
    if model_tag1:
        model_values=pc.find("div", class_="product-desc-sub-text")
    else:
        model_values=pc.find("div",class_="product-desc-sub-container")
    
    model = f"{model_tag.text.strip() if model_tag else ''} {model_values.text.strip() if model_tag1 else ''}".strip()
    models.append(model if model else np.nan)
    
    
    price_tag_div = pc.find("div", class_="prc-box-dscntd")
    if price_tag_div:
        price_value = price_tag_div.text.strip()
    else:
        
        price_tag_h6 = pc.find("span", class_="original-price")
        if price_tag_h6 and price_tag_h6.find("h6"):
            price_value = price_tag_h6.find("h6").text.strip()
        else:
            price_value = np.nan  
            
    prices.append(price_value)
    
    rating_score_tag = pc.find('span', class_='rating-score')
    rating_score = rating_score_tag.text.strip() if rating_score_tag else 0
    rating.append(rating_score)

   
    fv_tag = pc.find("span", class_="focused-text")
    fv = fv_tag.text.strip() if fv_tag else 0
    favorite_number.append(fv)

    rush_delivery_elem = pc.find('div', class_='product-stamp-container rushDelivery')
    rush_delivery_tag = 1 if rush_delivery_elem else 0
    rush_delivery.append(rush_delivery_tag)

    free_cargo_elem = pc.find('div', class_="product-stamp-container freeCargo")
    free_cargo_tag = 1 if free_cargo_elem else 0
    Free_cargo.append(free_cargo_tag)

  
    link_tag = pc.find('a', href=True)
    link = link_tag['href'] if link_tag else np.nan
    full_link = f"{link}" if link != np.nan else np.nan
    links.append(full_link)
    
df4 = pd.DataFrame({
    'Brand': brands,
    'Model': models,
    'Price': prices,  
    'Rating': rating,
    'Link': links,
    'Favori Number': favorite_number,
    'Rush Delivery': rush_delivery,
    'Free Cargo': Free_cargo
})


df4

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8.750 TL,4.2,https://www.trendyol.com/casper/nirvana-c370-4...,60.5B kişi,0,1
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13.499 TL,4.5,https://www.trendyol.com/lenovo/nb-v15-82yu012...,18.5B kişi,1,1
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18.299 TL,4.6,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,30.3B kişi,1,1
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11.005 TL,3.8,https://www.trendyol.com/lenovo/ideapad3-intel...,4354 kişi,1,1
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15.498 TL,4.5,https://www.trendyol.com/asus/vivobook-15-inte...,7974 kişi,1,1
...,...,...,...,...,...,...,...,...
5611,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43.399 TL,0,https://www.trendyol.com/lenovo/thinkpad-t14-g...,0,1,1
5612,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...","75.716,13 TL",0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0,0,1
5613,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...","39.655,09 TL",0,https://www.trendyol.com/hp/probook-450-g9-i5-...,0,0,1
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,"24.464,30 TL",5.0,https://www.trendyol.com/lenovo/v15-g3-intel-c...,0,0,1


In [34]:
data=df4.copy()

In [35]:
df4.isnull().sum()

Brand            0
Model            0
Price            0
Rating           0
Link             0
Favori Number    0
Rush Delivery    0
Free Cargo       0
dtype: int64

In [38]:
def get_product_details(product_url):
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    response = requests.get(product_url, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')

        # Ürün özelliklerini çekme
    features = {}

    for item in soup.find_all('li', class_='detail-attr-item'):
        key_elem = item.find('span', class_='attr-name attr-key-name-w')
        value_elem = item.find('div', class_='attr-name attr-value-name-w')

        if key_elem and value_elem:
            key = key_elem.text.strip()
            value = value_elem.text.strip()
            features[key] = value

    return features


In [41]:
product_data=[]
for link in df4['Link']:
    detail=get_product_details(link)
    if detail:
        product_data.append(detail)

df_details = pd.DataFrame(product_data)
df_details

,İşlemci Tipi,Ram (Sistem Belleği),SSD Kapasitesi,İşletim Sistemi,Ekran Kartı,Ekran Boyutu,Çözünürlük,Kullanım Amacı,Cihaz Ağırlığı,Ekran Yenileme Hızı,...,Power Supply,HDMI,Şarjlı Kullanım Süresi,Soket Tipi,SSD Teknolojisi,Beden,Okula Dönüş,Ekran Kartı Gücü,Görüntü Kalitesi,Ekran Özelliği
0,Intel Celeron,4 GB,120 GB,Windows,Intel UHD Graphics 600,"15,6 inç",1366 x 768,Ofis - İş,2 kg ve altı,60 Hz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMD Ryzen 5,16 GB,512 GB,Free Dos,Dahili Ekran Kartı,"15,6 inç",1920 x 1080,Ofis - İş,2 - 4 kg,Belirtilmemiş,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMD Ryzen 5,32 GB,512 GB,Free Dos,AMD Radeon RX 550,"24"" / 61 Ekran",1920 x 1080,NaN,4 kg ve üzeri,75 Hz,...,500 W ve üstü,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Intel Core i3,8 GB,256 GB,Free Dos,Intel UHD Graphics,"15,6 inç",1920 x 1080,Ofis - İş,2 kg ve altı,Belirtilmemiş,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Intel Core i5,8 GB,512 GB,Free Dos,Intel Iris Graphics,"15,6 inç",1920 x 1080,Ofis - İş,2 kg ve altı,60 Hz,...,NaN,NaN,4-6 Saat,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,AMD Ryzen 7,48 GB,2 TB,Windows,AMD Radeon Graphics,14 inç,1920 x 1080,Ofis - İş,2 kg ve altı,60 Hz,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5612,AMD Ryzen 7,24 GB,4 TB,Free Dos,Nvidia GeForce RTX 4070,"16,1 inç",1920 x 1080,Oyun,2 - 4 kg,144 Hz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5613,Intel Core i5,64 GB,256 GB,Windows,Intel XE Graphics,"15,6 inç",1920 x 1080,Ofis - İş,2 kg ve altı,60 Hz,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5614,Intel Core i7,24 GB,1 TB,Free Dos,Intel XE Graphics,"15,6 inç",1920 x 1080,Ofis - İş,2 kg ve altı,60 Hz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df=df4.join(df_details, how='outer')
df

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Power Supply,HDMI,Şarjlı Kullanım Süresi,Soket Tipi,SSD Teknolojisi,Beden,Okula Dönüş,Ekran Kartı Gücü,Görüntü Kalitesi,Ekran Özelliği
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8.750 TL,4.2,https://www.trendyol.com/casper/nirvana-c370-4...,60.5B kişi,0,1,Intel Celeron,4 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13.499 TL,4.5,https://www.trendyol.com/lenovo/nb-v15-82yu012...,18.5B kişi,1,1,AMD Ryzen 5,16 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18.299 TL,4.6,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,30.3B kişi,1,1,AMD Ryzen 5,32 GB,...,500 W ve üstü,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11.005 TL,3.8,https://www.trendyol.com/lenovo/ideapad3-intel...,4354 kişi,1,1,Intel Core i3,8 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15.498 TL,4.5,https://www.trendyol.com/asus/vivobook-15-inte...,7974 kişi,1,1,Intel Core i5,8 GB,...,NaN,NaN,4-6 Saat,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43.399 TL,0,https://www.trendyol.com/lenovo/thinkpad-t14-g...,0,1,1,AMD Ryzen 7,48 GB,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5612,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...","75.716,13 TL",0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0,0,1,AMD Ryzen 7,24 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5613,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...","39.655,09 TL",0,https://www.trendyol.com/hp/probook-450-g9-i5-...,0,0,1,Intel Core i5,64 GB,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,"24.464,30 TL",5.0,https://www.trendyol.com/lenovo/v15-g3-intel-c...,0,0,1,Intel Core i7,24 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df.to_csv('Trendyol_Computer_Price.csv', index=False, encoding='utf-8-sig')

In [131]:
df=pd.read_csv("Trendyol_Computer_Price.csv")
df

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Power Supply,HDMI,Şarjlı Kullanım Süresi,Soket Tipi,SSD Teknolojisi,Beden,Okula Dönüş,Ekran Kartı Gücü,Görüntü Kalitesi,Ekran Özelliği
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8.750 TL,4.2,https://www.trendyol.com/casper/nirvana-c370-4...,60.5B kişi,0,1,Intel Celeron,4 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13.499 TL,4.5,https://www.trendyol.com/lenovo/nb-v15-82yu012...,18.5B kişi,1,1,AMD Ryzen 5,16 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18.299 TL,4.6,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,30.3B kişi,1,1,AMD Ryzen 5,32 GB,...,500 W ve üstü,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11.005 TL,3.8,https://www.trendyol.com/lenovo/ideapad3-intel...,4354 kişi,1,1,Intel Core i3,8 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15.498 TL,4.5,https://www.trendyol.com/asus/vivobook-15-inte...,7974 kişi,1,1,Intel Core i5,8 GB,...,NaN,NaN,4-6 Saat,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43.399 TL,0.0,https://www.trendyol.com/lenovo/thinkpad-t14-g...,0,1,1,AMD Ryzen 7,48 GB,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5612,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...","75.716,13 TL",0.0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0,0,1,AMD Ryzen 7,24 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5613,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...","39.655,09 TL",0.0,https://www.trendyol.com/hp/probook-450-g9-i5-...,0,0,1,Intel Core i5,64 GB,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,"24.464,30 TL",5.0,https://www.trendyol.com/lenovo/v15-g3-intel-c...,0,0,1,Intel Core i7,24 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df.isnull().sum()

Brand                             0
Model                             4
Price                             0
Rating                            0
Link                              0
Favori Number                     0
Rush Delivery                     0
Free Cargo                        0
İşlemci Tipi                     22
Ram (Sistem Belleği)             20
SSD Kapasitesi                   20
İşletim Sistemi                   2
Ekran Kartı                       2
Ekran Boyutu                     30
Çözünürlük                       24
Kullanım Amacı                   92
Cihaz Ağırlığı                   10
Ekran Yenileme Hızı              44
Ekran Kartı Tipi                  2
Ekran Kartı Hafızası             10
İşlemci Modeli                   92
İşlemci Nesli                    92
İşlemci Çekirdek Sayısı           2
Hard Disk Kapasitesi             18
Dokunmatik Ekran                 92
Panel Tipi                     1656
Ram (Sistem Belleği) Tipi        12
Renk                        

In [173]:
def column_summary_table(dataframe, column_name):
    
    
    if column_name not in dataframe.columns:
        raise ValueError(f"Sütun adı '{column_name}' veri setinde bulunamadı.")
    
    
    column = dataframe[column_name]
    
    
    nan_count = column.isna().sum()
    nunique_count = column.nunique()
    value_counts = column.value_counts().to_dict()
    dtype = column.dtype
    describe = column.describe(include='all').to_dict()
    
   
    summary_data = [
        {"Özellik": "Sütun Adı", "Değer": column_name},
        {"Özellik": "Veri Türü", "Değer": str(dtype)},
        {"Özellik": "NaN Değer Sayısı", "Değer": nan_count},
        {"Özellik": "Benzersiz Değer Sayısı", "Değer": nunique_count},
        {"Özellik": "Değer Sayımları", "Değer": value_counts},
    ]
    
    
    for stat_name, stat_value in describe.items():
        summary_data.append({"Özellik": f"Dağılım - {stat_name}", "Değer": stat_value})
    
    
    
    
    summary_table = pd.DataFrame(summary_data)
    
    return summary_table

In [174]:
column_summary_table(df,'İşlemci Tipi')

,Özellik,Değer
0,Sütun Adı,İşlemci Tipi
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,47
4,Değer Sayımları,"{'Intel Core i5': 1784, 'Intel Core i7': 1444,..."
5,Dağılım - count,5616
6,Dağılım - unique,47
7,Dağılım - top,Intel Core i5
8,Dağılım - freq,1784


In [170]:
data=df[df['İşlemci Tipi'].isna()]
data

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Power Supply,HDMI,Şarjlı Kullanım Süresi,Soket Tipi,SSD Teknolojisi,Beden,Okula Dönüş,Ekran Kartı Gücü,Görüntü Kalitesi,Ekran Özelliği


In [55]:
column_summary_table(df,'Model')

,Özellik,Değer
0,Sütun Adı,Model
1,Veri Türü,object
2,NaN Değer Sayısı,4
3,Benzersiz Değer Sayısı,2646
4,Değer Sayımları,"{'512GB SSD RTX3050/6GB 15.6"" FHD 144Hz W11H 8..."
5,Dağılım - count,5612
6,Dağılım - unique,2646
7,Dağılım - top,"512GB SSD RTX3050/6GB 15.6"" FHD 144Hz W11H 83D..."
8,Dağılım - freq,16


In [85]:
df[df['Model'].isna()]

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Power Supply,HDMI,Şarjlı Kullanım Süresi,Soket Tipi,SSD Teknolojisi,Beden,Okula Dönüş,Ekran Kartı Gücü,Görüntü Kalitesi,Ekran Özelliği
2604,HP,NaN,"42.999,99 TL",5.0,https://www.trendyol.com/hp/n-a-p-675176564?bo...,656 kişi,0,1,Intel Core i7,16 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5412,HP,NaN,"42.999,99 TL",5.0,https://www.trendyol.com/hp/n-a-p-675176564?bo...,656 kişi,0,1,Intel Core i7,16 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
df.loc[(df['Brand'] == 'Hp')    & (df['Model'].isna()),'Model']='HP Victus 15'
df.loc[(df['Brand'] =='Huawei') & (df['Model'].isna()),'Model']='Huawei Matebook D15'

In [133]:
df[df['Model'].isna()]

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Power Supply,HDMI,Şarjlı Kullanım Süresi,Soket Tipi,SSD Teknolojisi,Beden,Okula Dönüş,Ekran Kartı Gücü,Görüntü Kalitesi,Ekran Özelliği
2604,HP,NaN,"42.999,99 TL",5.0,https://www.trendyol.com/hp/n-a-p-675176564?bo...,656 kişi,0,1,Intel Core i7,16 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5412,HP,NaN,"42.999,99 TL",5.0,https://www.trendyol.com/hp/n-a-p-675176564?bo...,656 kişi,0,1,Intel Core i7,16 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
df['Model']=df['Model'].fillna('HP Victus 15')

In [135]:
column_summary_table(df,'Model')

,Özellik,Değer
0,Sütun Adı,Model
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,2648
4,Değer Sayımları,"{'512GB SSD RTX3050/6GB 15.6"" FHD 144Hz W11H 8..."
5,Dağılım - count,5616
6,Dağılım - unique,2648
7,Dağılım - top,"512GB SSD RTX3050/6GB 15.6"" FHD 144Hz W11H 83D..."
8,Dağılım - freq,16


In [98]:
dtx=df[df['İşlemci Tipi'].isna()]

In [136]:
import re
def extract_processor(model):
    if not isinstance(model, str):
        return None
    match = re.search(r'\b(M2|M3|RYZEN[3-9]|CPU|8cpu|10cpu|i[3-9]|AMD)\b', model, re.IGNORECASE)
    return match.group(0) if match else None

# İşlemci Tipi sütununu güncelle
df['İşlemci Tipi'] = df.apply(
    lambda row: extract_processor(row['Model']) if pd.isna(row['İşlemci Tipi']) else row['İşlemci Tipi'],
    axis=1
)

In [137]:
column_summary_table(df,'İşlemci Tipi')

,Özellik,Değer
0,Sütun Adı,İşlemci Tipi
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,47
4,Değer Sayımları,"{'Intel Core i5': 1784, 'Intel Core i7': 1444,..."
5,Dağılım - count,5616
6,Dağılım - unique,47
7,Dağılım - top,Intel Core i5
8,Dağılım - freq,1784


In [138]:
df[df['Brand']=='Apple']['İşlemci Tipi']

55          Apple M1
91                M2
172               M2
176         Apple M2
194         Apple M1
            ...     
5131        Apple M3
5134        Apple M3
5193    Apple M4 Pro
5413        Apple M3
5568    Apple M4 Pro
Name: İşlemci Tipi, Length: 192, dtype: object

In [143]:
def fill_ssd(model, current_ssd):
    if pd.isna(current_ssd):  # Sadece NaN olanları doldur
        model_lower = model.lower()
        # "256GB SSD" kontrolü
        if "256gb ssd" in model_lower:
            return "256GB SSD"
        # "512GB SSD" kontrolü
        elif "512gb ssd" in model_lower:
            return "512GB SSD"
        # "2TBSSD+2TBSSD" kontrolü
        elif "2tbssd+2tbssd" in model_lower:
            return "2TBSSD+2TBSSD"
    return current_ssd

df['SSD Kapasitesi'] = df.apply(lambda row: fill_ssd(row['Model'], row['SSD Kapasitesi']), axis=1)

In [158]:
column_summary_table(df,'SSD Kapasitesi')

,Özellik,Değer
0,Sütun Adı,SSD Kapasitesi
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,18
4,Değer Sayımları,"{'512 GB': 2248, '1 TB': 1586, '256 GB': 750, ..."
5,Dağılım - count,5616
6,Dağılım - unique,18
7,Dağılım - top,512 GB
8,Dağılım - freq,2248


In [150]:
def fill_ssd(model, current_ssd):
    if pd.isna(current_ssd):  
        
        match = re.search(r'\b(\d+GB|\d+TB)( Ssd)?\b', model, re.IGNORECASE)

        if match:
            return match.group(1)  
    return current_ssd  


df['SSD Kapasitesi'] = df.apply(lambda row: fill_ssd(row['Model'], row['SSD Kapasitesi']), axis=1)

In [153]:
df.loc[(df['Model'] == 'Macbook Air 13.6" M2 8c Cpu 10c Gpu 8 Gb 512 Gb Ssd Gece Yarısı') & (df['SSD Kapasitesi'].isna()),'SSD Kapasitesi']='512 GB'
df.loc[(df['Model'] =='Macbook Air 13.6" M2 8c Gpu 8 Gb 256 Gb Ssd Yıldız Işığı ') & (df['SSD Kapasitesi'].isna()),'SSD Kapasitesi']='256 GB'

In [157]:
df.loc[(df['Model'] == 'Macbook Air 13.6" M2 8c Gpu 8 Gb 256 Gb Ssd Yıldız Işığı') & (df['SSD Kapasitesi'].isna()),'SSD Kapasitesi']='256 GB'
df.loc[(df['Model'] == 'Macbook Air 13.6" M2 8c Cpu 10c Gpu 8 Gb 512 Gb Ssd Uzay Grisi') & (df['SSD Kapasitesi'].isna()),'SSD Kapasitesi']='512 GB'

In [163]:
column_summary_table(df,'Ram (Sistem Belleği)')

,Özellik,Değer
0,Sütun Adı,Ram (Sistem Belleği)
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,17
4,Değer Sayımları,"{'16 GB': 2028, '8 GB': 1244, '32 GB': 1022, '..."
5,Dağılım - count,5616
6,Dağılım - unique,17
7,Dağılım - top,16 GB
8,Dağılım - freq,2028


In [162]:
df['Ram (Sistem Belleği)']=df['Ram (Sistem Belleği)'].fillna('8 GB')

In [165]:
column_summary_table(df,'İşletim Sistemi')

,Özellik,Değer
0,Sütun Adı,İşletim Sistemi
1,Veri Türü,object
2,NaN Değer Sayısı,2
3,Benzersiz Değer Sayısı,7
4,Değer Sayımları,"{'Free Dos': 2744, 'Windows': 2460, 'Linux': 2..."
5,Dağılım - count,5614
6,Dağılım - unique,7
7,Dağılım - top,Free Dos
8,Dağılım - freq,2744


In [167]:
df['İşletim Sistemi']=df['İşletim Sistemi'].fillna('Free Dos')

In [181]:
column_summary_table(df,'Ekran Kartı')

,Özellik,Değer
0,Sütun Adı,Ekran Kartı
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,81
4,Değer Sayımları,"{'Dahili Ekran Kartı': 814, 'Nvidia GeForce RT..."
5,Dağılım - count,5616
6,Dağılım - unique,81
7,Dağılım - top,Dahili Ekran Kartı
8,Dağılım - freq,814


In [180]:
df['Ekran Kartı']=df['Ekran Kartı'].fillna('Dahili Ekran Kartı')

In [196]:
column_summary_table(df,'Ekran Boyutu')

,Özellik,Değer
0,Sütun Adı,Ekran Boyutu
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,28
4,Değer Sayımları,"{'15,6 inç': 3440, '16 inç': 574, '14 inç': 50..."
5,Dağılım - count,5616
6,Dağılım - unique,28
7,Dağılım - top,"15,6 inç"
8,Dağılım - freq,3440


In [188]:
df.loc[(df['Model'] == '16GB DDR4 512GB NVMe Radeon RX 7600 24 inç Monitörlü Gaming Hazır Sistem') & (df['Ekran Boyutu'].isna()),'Ekran Boyutu']='24 inç'

In [190]:
df.loc[(df['Model'] == 'NB V15 82YU00P0TX RYZEN3-7320U 8GB 256SSD O/B 15.6 DOS') & (df['Ekran Boyutu'].isna()),'Ekran Boyutu']='15,6 inç'

In [193]:
df['Ekran Boyutu']=df['Ekran Boyutu'].fillna('15,6 inç')

In [225]:
column_summary_table(df,'Kullanım Amacı')

,Özellik,Değer
0,Sütun Adı,Kullanım Amacı
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,5
4,Değer Sayımları,"{'Oyun': 2642, 'Ofis - İş': 2228, 'Ev - Okul':..."
5,Dağılım - count,5616
6,Dağılım - unique,5
7,Dağılım - top,Oyun
8,Dağılım - freq,2642


In [223]:
df['Kullanım Amacı']=df['Kullanım Amacı'].fillna('Ofis - İş')

In [218]:
df.loc[(df['Brand']=='Zeiron')&(df['Kullanım Amacı'].isna()),'Kullanım Amacı']='Oyun'
df.loc[(df['Brand']=='Creature')&(df['Kullanım Amacı'].isna()),'Kullanım Amacı']='Oyun'

In [201]:
dfx=df.copy()

In [253]:
column_summary_table(df,'Cihaz Ağırlığı')

,Özellik,Değer
0,Sütun Adı,Cihaz Ağırlığı
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,4
4,Değer Sayımları,"{'2 - 4 kg': 2892, '2 kg ve altı': 2468, 'Beli..."
5,Dağılım - count,5616
6,Dağılım - unique,4
7,Dağılım - top,2 - 4 kg
8,Dağılım - freq,2892


In [242]:
df.loc[(df['Brand']=='LENOVO')&(df['Cihaz Ağırlığı'].isna()),'Cihaz Ağırlığı']='2 kg ve altı'

In [251]:
df['Cihaz Ağırlığı']=df['Cihaz Ağırlığı'].fillna('4 kg ve üzeri')

In [254]:
column_summary_table(df,'Ekran Yenileme Hızı')

,Özellik,Değer
0,Sütun Adı,Ekran Yenileme Hızı
1,Veri Türü,object
2,NaN Değer Sayısı,44
3,Benzersiz Değer Sayısı,10
4,Değer Sayımları,"{'144 Hz': 2004, '60 Hz': 1620, 'Belirtilmemiş..."
5,Dağılım - count,5572
6,Dağılım - unique,10
7,Dağılım - top,144 Hz
8,Dağılım - freq,2004


Convert to Numeric Values

In [292]:
df['Price']=df['Price'].str.replace('TL','',regex=False).str.replace('.','').str.replace(',', '.', regex=False).astype(float)
df['Price']

0        8750.00
1       13499.00
2       18299.00
3       11005.00
4       15498.00
          ...   
5611    43399.00
5612    75716.13
5613    39655.09
5614    24464.30
5615    24999.99
Name: Price, Length: 5616, dtype: float64

In [300]:
df['Ram (Sistem Belleği)']

Ram (Sistem Belleği)
16 GB     2028
8 GB      1244
32 GB     1022
64 GB      290
24 GB      242
96 GB      178
4 GB       160
48 GB      140
40 GB      112
12 GB       66
36 GB       60
20 GB       36
1 GB        20
18 GB       10
2 GB         4
80 GB        2
128 GB       2
Name: count, dtype: int64

In [305]:
def convert_to_gb(value):
    if isinstance(value,str):
        value=value.lower().strip()
        if 'gb' in value:
            value=value.replace('gb','').strip()
            return float(value)
    return value

In [310]:
df['Ram (Sistem Belleği)'].apply(convert_to_gb)

0        4.0
1       16.0
2       32.0
3        8.0
4        8.0
        ... 
5611    48.0
5612    24.0
5613    64.0
5614    24.0
5615    64.0
Name: Ram (Sistem Belleği), Length: 5616, dtype: float64

In [317]:
def convert_to_ssd_(value):
    if isinstance(value,str):
        value=value.lower().strip()
        if 'tbssd+' in value:
            value=value.replace('tbssd','000').replace('+','').strip()
            return value
    return value

In [321]:
df['SSD Kapasitesi']=df['SSD Kapasitesi'].apply(convert_to_ssd_)

In [325]:
a='20002000'
a=float(a.replace('20002000','4000').strip())
a


4000.0

In [338]:
def convert_to_ssd(values):
    if isinstance(values,str):
        values=values.lower().strip()
        if 'gb' in values:
            values=values.replace('gb','').strip()
            if 'ssd' in values:
                values=values.replace('ssd','').strip()
                return float(values)/1024
            else:
                return float(values)/1024
        
        elif 'yok' in values:
            return 0.0
        
        elif 'tb' in values:
            values=values.replace('tb','').strip()
            return float(values)
        
        elif '2000' in values:
            values=values.replace('20002000','4').strip()
            return float(values)
    

In [330]:
dfy=df.copy()

In [339]:
df['SSD Kapasitesi']=df['SSD Kapasitesi'].apply(convert_to_ssd)

In [347]:
def convert_to_hz(value):
    if isinstance(value,str):
        value=value.lower().strip()
        if 'hz' in value:
            value=value.replace('hz','').strip()
            return float(value)
        elif 'Belirtilmemiş' in value:
            value=value.replace('Belirtilmemiş','NaN').strip()
            value=np.nan
            return value
    

In [350]:
df['Ekran Yenileme Hızı']=df['Ekran Yenileme Hızı'].apply(convert_to_hz)
df['Ekran Yenileme Hızı'].value_counts()

Ekran Yenileme Hızı
144.0    2004
60.0     1620
120.0     374
165.0     256
240.0     234
90.0       94
75.0       18
360.0      16
300.0       8
Name: count, dtype: int64

In [355]:
column_summary_table(df,'Ekran Kartı Hafızası')

,Özellik,Değer
0,Sütun Adı,Ekran Kartı Hafızası
1,Veri Türü,object
2,NaN Değer Sayısı,10
3,Benzersiz Değer Sayısı,10
4,Değer Sayımları,"{'Paylaşımlı': 2670, '8 GB': 1052, '6 GB': 960..."
5,Dağılım - count,5606
6,Dağılım - unique,10
7,Dağılım - top,Paylaşımlı
8,Dağılım - freq,2670


In [364]:
def convert_to_ek(value):
    if isinstance(value,str):
        value=value.lower().strip()
        
        if 'gb' in value:
            value=value.replace('gb','').strip()
            if 've altı' in value:
                value=value.replace('ve altı','').strip()
                return float(value)
            else:
                return float(value)
        
        elif 'paylaşımlı' in value:
            value=value.replace('paylaşımlı','1').strip()
            return float(value)
        
        elif 'paylaşımsız' in value:
            value=value.replace('paylaşımsız','0').strip()
            return float(value)
        
        elif 'belirtilmemiş' in value:
            value=value.replace('belirtilmemiş','NaN').strip()
            value=np.nan
            return value

            

In [368]:
df['Ekran Kartı Hafızası']=df['Ekran Kartı Hafızası'].apply(convert_to_ek)
df['Ekran Kartı Hafızası'].value_counts()

Ekran Kartı Hafızası
1.0     2670
8.0     1052
6.0      960
4.0      748
16.0      70
12.0      50
10.0      42
32.0       6
0.0        2
Name: count, dtype: int64

In [375]:
column_summary_table(df,'İşlemci Çekirdek Sayısı')

,Özellik,Değer
0,Sütun Adı,İşlemci Çekirdek Sayısı
1,Veri Türü,object
2,NaN Değer Sayısı,2
3,Benzersiz Değer Sayısı,12
4,Değer Sayımları,"{'8': 1406, '10': 1170, '6': 622, '14': 518, '..."
5,Dağılım - count,5614
6,Dağılım - unique,12
7,Dağılım - top,8
8,Dağılım - freq,1406


In [376]:
def convert_to_ce(value):
    if isinstance(value,str):
        value=value.lower().strip()
        if '+' in value:
            value=value.replace('+','').strip()
            return float(value)
        elif 'belirtilmemiş' in value:
            value=np.nan
            return value
        else:
            return float(value)
        
         

In [380]:
df['İşlemci Çekirdek Sayısı']=df['İşlemci Çekirdek Sayısı'].apply(convert_to_ce)

In [381]:
column_summary_table(df,'Hard Disk Kapasitesi')

,Özellik,Değer
0,Sütun Adı,Hard Disk Kapasitesi
1,Veri Türü,object
2,NaN Değer Sayısı,18
3,Benzersiz Değer Sayısı,12
4,Değer Sayımları,"{'HDD Yok': 5064, '512 GB': 174, '500 GB': 112..."
5,Dağılım - count,5598
6,Dağılım - unique,12
7,Dağılım - top,HDD Yok
8,Dağılım - freq,5064


In [383]:
def convert_to_hdd(values):
    if isinstance(values,str):
        values=values.lower().strip()
        if 'gb' in values:
            values=values.replace('gb','').strip()
            if 'hdd' in values:
                values=values.replace('ssd','').strip()
                return float(values)/1024
            else:
                return float(values)/1024
        
        elif 'yok' in values:
            return 0.0
        
        elif 'tb' in values:
            values=values.replace('tb','').strip()
            return float(values)
        
        elif 'belirtilmemiş' in values:
            values=np.nan
            return float(values)
    

In [387]:
df['Hard Disk Kapasitesi']=df['Hard Disk Kapasitesi'].apply(convert_to_hdd)

In [388]:
df['Hard Disk Kapasitesi'].value_counts()

Hard Disk Kapasitesi
0.000000    5064
0.500000     174
0.488281     112
1.000000     102
0.250000      78
0.125000      18
0.244141      16
2.000000      14
0.031250       4
6.000000       2
0.732422       2
Name: count, dtype: int64

In [389]:
column_summary_table(df,'Temel İşlemci Hızı (GHz)')

,Özellik,Değer
0,Sütun Adı,Temel İşlemci Hızı (GHz)
1,Veri Türü,object
2,NaN Değer Sayısı,3110
3,Benzersiz Değer Sayısı,37
4,Değer Sayımları,"{'1.8': 298, '3.3': 202, '1.3': 182, '1.7': 14..."
5,Dağılım - count,2506
6,Dağılım - unique,37
7,Dağılım - top,1.8
8,Dağılım - freq,298


In [ ]:
dfy['Temel İşlemci Hızı (GHz)']=dfy['Temel İşlemci Hızı (GHz)'].apply(convert_to_ce)

In [404]:
def convert_to_is(value):
    if isinstance(value,str):
        value=value.lower().strip()
        if '2 - 3' in value:
            value=value.replace('-','').replace('2 ','2.').replace(' 3','3').strip()
            return float(value)
        elif '1 - 2' in value:
            value=value.replace('-','').replace('1 ','1.').replace(' 2','2').strip()
            return float(value)
        elif 'belirtilmemiş' in value:
            value=np.nan
            return value
        else:
            return float(value)

In [410]:
df['Temel İşlemci Hızı (GHz)']=df['Temel İşlemci Hızı (GHz)'].apply(convert_to_is)
df['Temel İşlemci Hızı (GHz)'].value_counts()

Temel İşlemci Hızı (GHz)
1.8    298
3.3    202
1.3    182
1.7    148
2.3    138
2.4    132
1.9    124
1.2    110
2.1    106
2.0    104
2.6     82
3.8     80
1.6     76
3.7     76
3.6     72
1.1     70
3.5     44
2.2     42
3.1     40
3.2     40
4.0     38
2.5     36
1.5     36
3.4     32
1.0     22
2.8     20
2.7     18
3.0     18
2.9     12
1.4     12
3.9     10
5.1      4
4.1      4
4.2      2
Name: count, dtype: int64

In [422]:
def convert_to_ga(value):
    if isinstance(value,str):
        value=value.lower().strip()
        if 'yıl' in value:
            value=value.replace('yıl','').strip()
            return int(value)


In [429]:
df['Garanti Süresi']=df['Garanti Süresi'].apply(convert_to_ga)

In [430]:
column_summary_table(df,'Garanti Süresi')

,Özellik,Değer
0,Sütun Adı,Garanti Süresi
1,Veri Türü,float64
2,NaN Değer Sayısı,16
3,Benzersiz Değer Sayısı,3
4,Değer Sayımları,"{2.0: 5244, 3.0: 216, 1.0: 140}"
5,Dağılım - count,5600.0
6,Dağılım - mean,2.013571
7,Dağılım - std,0.251791
8,Dağılım - min,1.0
9,Dağılım - 25%,2.0


In [431]:
column_summary_table(df,'Arttırılabilir Azami Bellek')

,Özellik,Değer
0,Sütun Adı,Arttırılabilir Azami Bellek
1,Veri Türü,object
2,NaN Değer Sayısı,3264
3,Benzersiz Değer Sayısı,8
4,Değer Sayımları,"{'64 GB': 1204, '16 GB': 386, '32 GB': 384, '8..."
5,Dağılım - count,2352
6,Dağılım - unique,8
7,Dağılım - top,64 GB
8,Dağılım - freq,1204


In [435]:
df['Arttırılabilir Azami Bellek']=df['Arttırılabilir Azami Bellek'].apply(convert_to_gb)
df['Arttırılabilir Azami Bellek'].value_counts()

Arttırılabilir Azami Bellek
64.0     1204
16.0      386
32.0      384
8.0       174
4.0        68
40.0       68
128.0      48
12.0       20
Name: count, dtype: int64

In [437]:
df=df.drop('Kapasite',axis=1)
df

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Power Supply,HDMI,Şarjlı Kullanım Süresi,Soket Tipi,SSD Teknolojisi,Beden,Okula Dönüş,Ekran Kartı Gücü,Görüntü Kalitesi,Ekran Özelliği
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8750.00,4.2,https://www.trendyol.com/casper/nirvana-c370-4...,60.5B kişi,0,1,Intel Celeron,4 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13499.00,4.5,https://www.trendyol.com/lenovo/nb-v15-82yu012...,18.5B kişi,1,1,AMD Ryzen 5,16 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18299.00,4.6,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,30.3B kişi,1,1,AMD Ryzen 5,32 GB,...,500 W ve üstü,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11005.00,3.8,https://www.trendyol.com/lenovo/ideapad3-intel...,4354 kişi,1,1,Intel Core i3,8 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15498.00,4.5,https://www.trendyol.com/asus/vivobook-15-inte...,7974 kişi,1,1,Intel Core i5,8 GB,...,NaN,NaN,4-6 Saat,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43399.00,0.0,https://www.trendyol.com/lenovo/thinkpad-t14-g...,0,1,1,AMD Ryzen 7,48 GB,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5612,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",75716.13,0.0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0,0,1,AMD Ryzen 7,24 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5613,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",39655.09,0.0,https://www.trendyol.com/hp/probook-450-g9-i5-...,0,0,1,Intel Core i5,64 GB,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,24464.30,5.0,https://www.trendyol.com/lenovo/v15-g3-intel-c...,0,0,1,Intel Core i7,24 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [440]:
df=df.drop('İşlemci Frekansı',axis=1)
df

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Power Supply,HDMI,Şarjlı Kullanım Süresi,Soket Tipi,SSD Teknolojisi,Beden,Okula Dönüş,Ekran Kartı Gücü,Görüntü Kalitesi,Ekran Özelliği
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8750.00,4.2,https://www.trendyol.com/casper/nirvana-c370-4...,60.5B kişi,0,1,Intel Celeron,4 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13499.00,4.5,https://www.trendyol.com/lenovo/nb-v15-82yu012...,18.5B kişi,1,1,AMD Ryzen 5,16 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18299.00,4.6,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,30.3B kişi,1,1,AMD Ryzen 5,32 GB,...,500 W ve üstü,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11005.00,3.8,https://www.trendyol.com/lenovo/ideapad3-intel...,4354 kişi,1,1,Intel Core i3,8 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15498.00,4.5,https://www.trendyol.com/asus/vivobook-15-inte...,7974 kişi,1,1,Intel Core i5,8 GB,...,NaN,NaN,4-6 Saat,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43399.00,0.0,https://www.trendyol.com/lenovo/thinkpad-t14-g...,0,1,1,AMD Ryzen 7,48 GB,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5612,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",75716.13,0.0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0,0,1,AMD Ryzen 7,24 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5613,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",39655.09,0.0,https://www.trendyol.com/hp/probook-450-g9-i5-...,0,0,1,Intel Core i5,64 GB,...,NaN,NaN,3 Saat ve Altı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,24464.30,5.0,https://www.trendyol.com/lenovo/v15-g3-intel-c...,0,0,1,Intel Core i7,24 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [443]:
df=df.drop('Power Supply',axis=1)
df.columns

Index(['Brand', 'Model', 'Price', 'Rating', 'Link', 'Favori Number',
       'Rush Delivery', 'Free Cargo', 'İşlemci Tipi', 'Ram (Sistem Belleği)',
       'SSD Kapasitesi', 'İşletim Sistemi', 'Ekran Kartı', 'Ekran Boyutu',
       'Çözünürlük', 'Kullanım Amacı', 'Cihaz Ağırlığı', 'Ekran Yenileme Hızı',
       'Ekran Kartı Tipi', 'Ekran Kartı Hafızası', 'İşlemci Modeli',
       'İşlemci Nesli', 'İşlemci Çekirdek Sayısı', 'Hard Disk Kapasitesi',
       'Dokunmatik Ekran', 'Panel Tipi', 'Ram (Sistem Belleği) Tipi', 'Renk',
       'Garanti Tipi', 'Temel İşlemci Hızı (GHz)',
       'Maksimum İşlemci Hızı (GHz)', 'Optik Sürücü Tipi', 'Bağlantılar',
       'Klavye', 'Ekran Kartı Bellek Tipi', 'Çözünürlük Standartı',
       'Garanti Süresi', 'Arttırılabilir Azami Bellek', 'Menşei', 'HDMI',
       'Şarjlı Kullanım Süresi', 'Soket Tipi', 'SSD Teknolojisi', 'Beden',
       'Okula Dönüş', 'Ekran Kartı Gücü', 'Görüntü Kalitesi',
       'Ekran Özelliği'],
      dtype='object')

In [444]:
column_summary_table(df,'HDMI')

,Özellik,Değer
0,Sütun Adı,HDMI
1,Veri Türü,object
2,NaN Değer Sayısı,5520
3,Benzersiz Değer Sayısı,1
4,Değer Sayımları,{'Var': 96}
5,Dağılım - count,96
6,Dağılım - unique,1
7,Dağılım - top,Var
8,Dağılım - freq,96


In [445]:
df['HDMI']=df['HDMI'].fillna('Yok')

In [446]:
column_summary_table(df,'Şarjlı Kullanım Süresi')

,Özellik,Değer
0,Sütun Adı,Şarjlı Kullanım Süresi
1,Veri Türü,object
2,NaN Değer Sayısı,4276
3,Benzersiz Değer Sayısı,3
4,Değer Sayımları,"{'4-6 Saat': 558, '3 Saat ve Altı': 498, '6 Sa..."
5,Dağılım - count,1340
6,Dağılım - unique,3
7,Dağılım - top,4-6 Saat
8,Dağılım - freq,558


In [451]:
df=df.drop('Şarjlı Kullanım Süresi',axis=1)
df

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Arttırılabilir Azami Bellek,Menşei,HDMI,Soket Tipi,SSD Teknolojisi,Beden,Okula Dönüş,Ekran Kartı Gücü,Görüntü Kalitesi,Ekran Özelliği
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8750.00,4.2,https://www.trendyol.com/casper/nirvana-c370-4...,60.5B kişi,0,1,Intel Celeron,4 GB,...,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13499.00,4.5,https://www.trendyol.com/lenovo/nb-v15-82yu012...,18.5B kişi,1,1,AMD Ryzen 5,16 GB,...,32.0,TR,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18299.00,4.6,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,30.3B kişi,1,1,AMD Ryzen 5,32 GB,...,64.0,NaN,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11005.00,3.8,https://www.trendyol.com/lenovo/ideapad3-intel...,4354 kişi,1,1,Intel Core i3,8 GB,...,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15498.00,4.5,https://www.trendyol.com/asus/vivobook-15-inte...,7974 kişi,1,1,Intel Core i5,8 GB,...,8.0,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43399.00,0.0,https://www.trendyol.com/lenovo/thinkpad-t14-g...,0,1,1,AMD Ryzen 7,48 GB,...,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5612,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",75716.13,0.0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0,0,1,AMD Ryzen 7,24 GB,...,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5613,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",39655.09,0.0,https://www.trendyol.com/hp/probook-450-g9-i5-...,0,0,1,Intel Core i5,64 GB,...,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,24464.30,5.0,https://www.trendyol.com/lenovo/v15-g3-intel-c...,0,0,1,Intel Core i7,24 GB,...,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [453]:
df=df.drop('Beden',axis=1)
df.columns

Index(['Brand', 'Model', 'Price', 'Rating', 'Link', 'Favori Number',
       'Rush Delivery', 'Free Cargo', 'İşlemci Tipi', 'Ram (Sistem Belleği)',
       'SSD Kapasitesi', 'İşletim Sistemi', 'Ekran Kartı', 'Ekran Boyutu',
       'Çözünürlük', 'Kullanım Amacı', 'Cihaz Ağırlığı', 'Ekran Yenileme Hızı',
       'Ekran Kartı Tipi', 'Ekran Kartı Hafızası', 'İşlemci Modeli',
       'İşlemci Nesli', 'İşlemci Çekirdek Sayısı', 'Hard Disk Kapasitesi',
       'Dokunmatik Ekran', 'Panel Tipi', 'Ram (Sistem Belleği) Tipi', 'Renk',
       'Garanti Tipi', 'Temel İşlemci Hızı (GHz)',
       'Maksimum İşlemci Hızı (GHz)', 'Optik Sürücü Tipi', 'Bağlantılar',
       'Klavye', 'Ekran Kartı Bellek Tipi', 'Çözünürlük Standartı',
       'Garanti Süresi', 'Arttırılabilir Azami Bellek', 'Menşei', 'HDMI',
       'Soket Tipi', 'SSD Teknolojisi', 'Okula Dönüş', 'Ekran Kartı Gücü',
       'Görüntü Kalitesi', 'Ekran Özelliği'],
      dtype='object')

In [454]:
column_summary_table(df,'Ekran Kartı Gücü')

,Özellik,Değer
0,Sütun Adı,Ekran Kartı Gücü
1,Veri Türü,object
2,NaN Değer Sayısı,5280
3,Benzersiz Değer Sayısı,15
4,Değer Sayımları,"{'140W': 86, '105W': 72, '95W': 32, '100W': 30..."
5,Dağılım - count,336
6,Dağılım - unique,15
7,Dağılım - top,140W
8,Dağılım - freq,86


In [468]:
df=df.drop('Ekran Kartı Gücü',axis=1)
df.columns

Index(['Brand', 'Model', 'Price', 'Rating', 'Link', 'Favori Number',
       'Rush Delivery', 'Free Cargo', 'İşlemci Tipi', 'Ram (Sistem Belleği)',
       'SSD Kapasitesi', 'İşletim Sistemi', 'Ekran Kartı', 'Ekran Boyutu',
       'Çözünürlük', 'Kullanım Amacı', 'Cihaz Ağırlığı', 'Ekran Yenileme Hızı',
       'Ekran Kartı Tipi', 'Ekran Kartı Hafızası', 'İşlemci Modeli',
       'İşlemci Nesli', 'İşlemci Çekirdek Sayısı', 'Hard Disk Kapasitesi',
       'Dokunmatik Ekran', 'Panel Tipi', 'Ram (Sistem Belleği) Tipi', 'Renk',
       'Garanti Tipi', 'Temel İşlemci Hızı (GHz)',
       'Maksimum İşlemci Hızı (GHz)', 'Optik Sürücü Tipi', 'Bağlantılar',
       'Klavye', 'Ekran Kartı Bellek Tipi', 'Çözünürlük Standartı',
       'Garanti Süresi', 'Arttırılabilir Azami Bellek', 'HDMI',
       'SSD Teknolojisi', 'Görüntü Kalitesi', 'Ekran Özelliği'],
      dtype='object')

In [470]:
df['Ekran Kartı Gücü']=dfx['Ekran Kartı Gücü']

In [ ]:
def convert_to_w(value):
    if isinstance(value,str):
        value=value.lower().strip()
        if 'w' in value:
            value=value.replace('w','').strip()
            return float(value)


0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
5611   NaN
5612   NaN
5613   NaN
5614   NaN
5615   NaN
Name: Ekran Kartı Gücü, Length: 5616, dtype: float64

In [487]:
df['Ekran Kartı Gücü']=df['Ekran Kartı Gücü'].apply(convert_to_w)
df['Ekran Kartı Gücü'].value_counts()

Ekran Kartı Gücü
140.0    86
105.0    72
95.0     32
100.0    30
75.0     20
115.0    20
60.0     18
55.0     16
45.0     14
175.0    14
90.0      4
70.0      4
135.0     2
150.0     2
120.0     2
Name: count, dtype: int64

In [461]:
df=df.drop('Okula Dönüş',axis=1)
df.columns

Index(['Brand', 'Model', 'Price', 'Rating', 'Link', 'Favori Number',
       'Rush Delivery', 'Free Cargo', 'İşlemci Tipi', 'Ram (Sistem Belleği)',
       'SSD Kapasitesi', 'İşletim Sistemi', 'Ekran Kartı', 'Ekran Boyutu',
       'Çözünürlük', 'Kullanım Amacı', 'Cihaz Ağırlığı', 'Ekran Yenileme Hızı',
       'Ekran Kartı Tipi', 'Ekran Kartı Hafızası', 'İşlemci Modeli',
       'İşlemci Nesli', 'İşlemci Çekirdek Sayısı', 'Hard Disk Kapasitesi',
       'Dokunmatik Ekran', 'Panel Tipi', 'Ram (Sistem Belleği) Tipi', 'Renk',
       'Garanti Tipi', 'Temel İşlemci Hızı (GHz)',
       'Maksimum İşlemci Hızı (GHz)', 'Optik Sürücü Tipi', 'Bağlantılar',
       'Klavye', 'Ekran Kartı Bellek Tipi', 'Çözünürlük Standartı',
       'Garanti Süresi', 'Arttırılabilir Azami Bellek', 'Menşei', 'HDMI',
       'Soket Tipi', 'SSD Teknolojisi', 'Ekran Kartı Gücü', 'Görüntü Kalitesi',
       'Ekran Özelliği'],
      dtype='object')

In [500]:
def convert_to_fav(value):
    if isinstance(value,str):
        value=value.lower().strip()
        if 'kişi' in value:
            value=value.replace('kişi','').strip()
            if 'b' in value:
                value=value.replace('b','').strip()
                return float(value)*1000
            else:
                return float(value)
        else:
            return float(value)

In [505]:
df['Favori Number']=df['Favori Number'].apply(convert_to_fav)
df['Favori Number'].unique()

array([6.050e+04, 1.850e+04, 3.030e+04, 4.354e+03, 7.974e+03, 2.273e+03,
       2.980e+04, 4.350e+04, 1.365e+03, 4.300e+04, 3.421e+03, 5.763e+03,
       7.350e+02, 1.380e+04, 4.440e+04, 9.201e+03, 3.690e+04, 1.141e+03,
       5.527e+03, 3.514e+03, 5.382e+03, 9.251e+03, 1.000e+03, 6.860e+02,
       1.012e+03, 1.643e+03, 1.130e+02, 1.688e+03, 2.300e+02, 2.271e+03,
       6.796e+03, 8.000e+02, 3.050e+03, 1.783e+03, 3.493e+03, 1.123e+03,
       1.230e+02, 1.170e+04, 4.280e+02, 5.960e+02, 6.561e+03, 1.410e+02,
       3.650e+03, 0.000e+00, 1.700e+03, 5.451e+03, 3.090e+02, 1.442e+03,
       1.642e+03, 5.170e+02, 3.722e+03, 4.235e+03, 1.162e+03, 1.500e+05,
       1.139e+03, 6.900e+02, 1.517e+03, 2.095e+03, 8.800e+02, 6.920e+02,
       1.650e+02, 4.013e+03, 1.011e+03, 4.400e+02, 5.000e+02, 2.480e+02,
       2.946e+03, 4.930e+02, 2.358e+03, 5.573e+03, 5.654e+03, 2.750e+02,
       2.465e+03, 4.040e+02, 9.940e+02, 1.420e+04, 1.407e+03, 2.219e+03,
       4.670e+02, 4.850e+02, 3.980e+02, 7.050e+02, 

In [519]:
column_summary_table(df,'İşlemci Nesli')

,Özellik,Değer
0,Sütun Adı,İşlemci Nesli
1,Veri Türü,object
2,NaN Değer Sayısı,92
3,Benzersiz Değer Sayısı,15
4,Değer Sayımları,"{'12. Nesil': 1648, '13. Nesil': 1432, '7. Nes..."
5,Dağılım - count,5524
6,Dağılım - unique,15
7,Dağılım - top,12. Nesil
8,Dağılım - freq,1648


In [524]:
df[df['İşlemci Nesli']=='7. Nesil']

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Klavye,Ekran Kartı Bellek Tipi,Çözünürlük Standartı,Garanti Süresi,Arttırılabilir Azami Bellek,HDMI,SSD Teknolojisi,Görüntü Kalitesi,Ekran Özelliği,Ekran Kartı Gücü
99,HP,8GB.RAM 512GB.SSD FDOS (Model:9G1G1ET),10776.90,0.0,https://www.trendyol.com/hp/255-15-6-r3-7330u-...,112.0,0,1,AMD Ryzen 3,8 GB,...,Q Türkçe,DDR4,Full HD (FHD),2.0,NaN,Yok,NaN,NaN,NaN,NaN
103,ASUS,Tuf Gamıng A15 R7 - 7735hs 32gb 1tb Ssd Rtx405...,48999.00,4.8,https://www.trendyol.com/asus/tuf-gaming-a15-r...,1400.0,1,1,AMD Ryzen 7,32 GB,...,Q Türkçe,GDDR6,Full HD (FHD),2.0,NaN,Yok,NaN,NaN,NaN,NaN
129,HP,Victus 15-fb2028nt Amd R5 7535hs 16gb 512gb Ss...,24999.00,4.3,https://www.trendyol.com/hp/victus-15-fb2028nt...,1538.0,1,1,AMD Ryzen 5,16 GB,...,Q Türkçe + Numerik (Aydınlatmalı),GDDR6,Full HD (FHD),2.0,32.0,Yok,NaN,NaN,NaN,NaN
152,ACER,"Aspire3 AMD Ryzen 5-7520U 8GB 512GB SSD 15.6"" ...",15999.00,4.4,https://www.trendyol.com/acer/aspire3-amd-ryze...,9168.0,0,1,AMD Ryzen 5,8 GB,...,Q Türkçe (Aydınlatmasız),Dahili,Full HD (FHD),2.0,NaN,Yok,NaN,NaN,NaN,NaN
167,LENOVO,V15 G4 AMN 82YU0124TX AMD Ryzen 5 7520U 8GB 51...,13299.00,4.9,https://www.trendyol.com/lenovo/v15-g4-amn-82y...,300.0,0,1,AMD Ryzen 5,8 GB,...,Q Türkçe,Dahili,Full HD (FHD),2.0,NaN,Yok,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5599,HP,"8GB 2TB SSD RTX4060/8GB 144HZ 16.1"" FHD W11H 7...",62279.50,0.0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0.0,0,1,AMD Ryzen 7,8 GB,...,Q Türkçe (Aydınlatmalı),DDR5,Full HD (FHD),2.0,NaN,Yok,NaN,NaN,NaN,NaN
5600,HP,"24GB 1TB SSD RTX4060/8GB 144HZ 16.1"" FHD W11H ...",59387.12,0.0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0.0,0,1,AMD Ryzen 7,24 GB,...,Q Türkçe (Aydınlatmalı),DDR5,Full HD (FHD),2.0,NaN,Yok,NaN,NaN,NaN,NaN
5601,HP,"24GB 2TB SSD RTX4060/8GB 144HZ 16.1"" FHD W11H ...",64435.07,0.0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0.0,0,1,AMD Ryzen 7,24 GB,...,Q Türkçe (Aydınlatmalı),DDR5,Full HD (FHD),2.0,NaN,Yok,NaN,NaN,NaN,NaN
5602,HP,"24GB 256GB SSD RTX4060/8GB 144HZ 16.1"" FHD W11...",60370.55,0.0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0.0,0,1,AMD Ryzen 7,24 GB,...,Q Türkçe (Aydınlatmalı),DDR5,Full HD (FHD),2.0,NaN,Yok,NaN,NaN,NaN,NaN


In [511]:
df['Dokunmatik Ekran']=df['Dokunmatik Ekran'].fillna('Yok')

In [513]:
df['Dokunmatik Ekran']=df['Dokunmatik Ekran'].replace('Belirtilmemiş','Yok')

In [529]:
# İşlemci Modeline göre eksik İşlemci Nesli'ni doldurma
dfy['İşlemci Nesli'] = dfy.groupby('İşletim Sistemi')['İşlemci Nesli'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x.mean()))


In [553]:
column_summary_table(df,'Ekran Kartı Bellek Tipi')

,Özellik,Değer
0,Sütun Adı,Ekran Kartı Bellek Tipi
1,Veri Türü,object
2,NaN Değer Sayısı,2
3,Benzersiz Değer Sayısı,13
4,Değer Sayımları,"{'GDDR6': 2400, 'Dahili': 1680, 'Belirtilmemiş..."
5,Dağılım - count,5614
6,Dağılım - unique,13
7,Dağılım - top,GDDR6
8,Dağılım - freq,2400


In [572]:
def Belirtilmemis(value):
    if isinstance(value, str):  
        if 'Belirtilmemiş' in value:  
            return np.nan
    return value

In [571]:
dfc=df.copy()

In [575]:
df['Ekran Kartı Bellek Tipi']=df['Ekran Kartı Bellek Tipi'].apply(Belirtilmemis)
df['Ekran Kartı Bellek Tipi'].value_counts()

Ekran Kartı Bellek Tipi
GDDR6     2400
Dahili    1680
DDR5       320
DDR4       270
SD         122
DDR         72
GDDR5       58
GDDR5X      58
DDR3        36
GDDR6X      36
DDR2         6
GDDR4        4
Name: count, dtype: int64

In [550]:
df['Garanti Tipi']=df['Garanti Tipi'].fillna('Lenovo Türkiye Garantili')

In [617]:
data=df.groupby(['Brand', 'Model'])['Ekran Kartı Bellek Tipi'].unique().reset_index()
data

,Brand,Model,Ekran Kartı Bellek Tipi
0,ACER,1 TB SSD RTX4060-8 GB 15.6'' 165Hz FreeDos NH....,[GDDR6]
1,ACER,"13420h 16gb 512gb Ssd Rtx 2050-4gb 15.6"" Fhd 1...",[GDDR6]
2,ACER,"13420h 32gb 512gb Ssd Rtx 2050-4gb 15.6"" Fhd 1...",[GDDR6]
3,ACER,"13420h 8gb 512gb Ssd Rtx2050-4gb 15.6"" Fhd 144...",[GDDR6]
4,ACER,15.6” FHD İntel İ3-N305 8 GB DDR5 RAM 256 GB S...,[Dahili]
...,...,...,...
2644,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,[DDR4]
2645,Zeiron,xTreme V2 Ryzen 7 5700X 32GB RAM 1TB M.2 Nvme ...,[GDDR6]
2646,icotech,DS16 16 inç 4K Çift Ekranlı i7 32 Gb Ram 1 Tb ...,[Dahili]
2647,icotech,İ7 12.Nesil 64Gb Ram 1 Tb Ssd Çift Ekranlı Dok...,[DDR4]


In [641]:
dfc=df.copy()

In [642]:
data=df.groupby(['Brand', 'Model'])['Ekran Kartı Bellek Tipi'].unique().reset_index()
for brand in df[df['Ekran Kartı Bellek Tipi'].isna()]['Brand'].unique():
    fillna_value=data[data['Brand']==brand]['Ekran Kartı Bellek Tipi'].mode()[0]
    dfc.loc[(dfc['Brand']==brand)&(dfc['Ekran Kartı Bellek Tipi'].isna()),'Ekran Kartı Bellek Tipi']=fillna_value

ValueError: Must have equal len keys and value when setting with an iterable

In [834]:
dfc.loc[(dfc['Brand']=='LENOVO')&(dfc['Cihaz Ağırlığı'].isna()),'Cihaz Ağırlığı']='2 kg ve altı'

In [833]:
dfc['Garanti Süresi']=dfc['Garanti Süresi'].fillna(2.0)

In [830]:
df.loc[(dfc['Brand']=='Creature')&(dfc['Klavye'].isna()),'Klavye']='Yok'
df.loc[(dfc['Brand']=='Zeiron')&(dfc['Klavye'].isna()),'Klavye']='Yok'


In [831]:
df.loc[(dfc['Brand']=='LENOVO')&(dfc['Klavye'].isna()),'Klavye']='Q Türkçe'

In [673]:
column_summary_table(df,'Ekran Kartı Tipi')

,Özellik,Değer
0,Sütun Adı,Ekran Kartı Tipi
1,Veri Türü,object
2,NaN Değer Sayısı,12
3,Benzersiz Değer Sayısı,2
4,Değer Sayımları,"{'Dahili': 2804, 'Harici': 2800}"
5,Dağılım - count,5604
6,Dağılım - unique,2
7,Dağılım - top,Dahili
8,Dağılım - freq,2804


In [840]:
dfc['Ekran Kartı Tipi']=dfc['Ekran Kartı Tipi'].apply(Belirtilmemis)

In [678]:
df.groupby(['Brand','Model'])['Ekran Kartı Tipi'].unique().reset_index()

,Brand,Model,Ekran Kartı Tipi
0,ACER,1 TB SSD RTX4060-8 GB 15.6'' 165Hz FreeDos NH....,[Harici]
1,ACER,"13420h 16gb 512gb Ssd Rtx 2050-4gb 15.6"" Fhd 1...",[Harici]
2,ACER,"13420h 32gb 512gb Ssd Rtx 2050-4gb 15.6"" Fhd 1...",[Harici]
3,ACER,"13420h 8gb 512gb Ssd Rtx2050-4gb 15.6"" Fhd 144...",[Dahili]
4,ACER,15.6” FHD İntel İ3-N305 8 GB DDR5 RAM 256 GB S...,[Harici]
...,...,...,...
2644,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,[Harici]
2645,Zeiron,xTreme V2 Ryzen 7 5700X 32GB RAM 1TB M.2 Nvme ...,[Harici]
2646,icotech,DS16 16 inç 4K Çift Ekranlı i7 32 Gb Ram 1 Tb ...,[Harici]
2647,icotech,İ7 12.Nesil 64Gb Ram 1 Tb Ssd Çift Ekranlı Dok...,[Dahili]


In [850]:
for marka in dfc['Brand'].unique():
    dx=df[df['Brand']=='ACER']['Ekran Kartı Tipi'].value_counts().reset_index()
    val=dx.loc[dx['count'].idxmax(), 'Ekran Kartı Tipi']
    df.loc[(dfc['Brand']==marka)&(df['Ekran Kartı Tipi'].isna()),'Ekran Kartı Tipi']=val
    

In [704]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

le_brand = LabelEncoder()
dfc['Brand_Encoded'] = le_brand.fit_transform(dfc['Brand'])

# KNN Imputer için veriyi hazırlayın
imputer = KNNImputer(n_neighbors=5)  # KNN komşu sayısı
features = ['Brand_Encoded']  # Daha fazla özellik ekleyebilirsiniz
dfc['Ekran Kartı Tipi Encoded'] = dfc['Ekran Kartı Tipi'].astype('category').cat.codes
imputed = imputer.fit_transform(dfc[features + ['Ekran Kartı Tipi Encoded']])
categories = dfc['Ekran Kartı Tipi'].astype('category').cat.categories
# Doldurulan veriyi orijinal kategoriye çevir
dfc['Ekran Kartı Tipi'] = pd.Categorical.from_codes(
    imputed[:, -1].astype(int),  # Doldurulan değerler
    categories=categories        # Orijinal kategoriler
)

In [719]:
column_summary_table(df,'Ram (Sistem Belleği) Tipi')

,Özellik,Değer
0,Sütun Adı,Ram (Sistem Belleği) Tipi
1,Veri Türü,object
2,NaN Değer Sayısı,94
3,Benzersiz Değer Sayısı,12
4,Değer Sayımları,"{'DDR4': 2782, 'DDR5': 2046, 'LPDDR5': 242, 'B..."
5,Dağılım - count,5522
6,Dağılım - unique,12
7,Dağılım - top,DDR4
8,Dağılım - freq,2782


In [718]:
df['Ram (Sistem Belleği) Tipi']=df['Ram (Sistem Belleği) Tipi'].apply(Belirtilmemis)

In [720]:
dfc['Ram (Sistem Belleği) Tipi']=dfc['Ram (Sistem Belleği) Tipi'].apply(Belirtilmemis)

In [724]:
le_brand = LabelEncoder()
dfc['Brand_Encoded'] = le_brand.fit_transform(dfc['Brand'])

# KNN Imputer için veriyi hazırlayın
imputer = KNNImputer(n_neighbors=5)  # KNN komşu sayısı
features = ['Brand_Encoded']  # Daha fazla özellik ekleyebilirsiniz
dfc['Ekran Kartı Tipi Encoded'] = dfc['Ekran Kartı Tipi'].astype('category').cat.codes
imputed = imputer.fit_transform(dfc[features + ['Ekran Kartı Tipi Encoded']])
categories = dfc['Ekran Kartı Tipi'].astype('category').cat.categories
# Doldurulan veriyi orijinal kategoriye çevir
dfc['Ekran Kartı Tipi'] = pd.Categorical.from_codes(
    imputed[:, -1].astype(int),  # Doldurulan değerler
    categories=categories        # Orijinal kategoriler
)

In [862]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier,XGBRFClassifier
from sklearn.linear_model import LogisticRegression,PassiveAggressiveClassifier,SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,HistGradientBoostingClassifier
from sklearn.metrics import mean_squared_error, classification_report, accuracy_score,f1_score,balanced_accuracy_score
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC



In [907]:
models = [
          ('LG', LogisticRegression()),
          ('KNN', KNeighborsClassifier()),
          ('CART', DecisionTreeClassifier()),
          ('HistgBM',HistGradientBoostingClassifier()),
          ('RF', RandomForestClassifier()),
          ("XGBoost",XGBClassifier()),
          ("XGBoostRF", XGBRFClassifier()),
          ("LightGBM", LGBMClassifier(verbose=-1))]

result={'Model':[],'Acc':[],'Acc_Balanced':[]}
for name, classifier in models:
    
    classifier.fit(x_tr, y_tr)
    y_pr=classifier.predict(x_ts)
    acc=accuracy_score(y_ts,y_pr)
    
    acc_balanced=balanced_accuracy_score(y_ts,y_pr)

    result['Model'].append(name)
    result['Acc'].append(acc)
    result['Acc_Balanced'].append(acc_balanced)
    

result=pd.DataFrame(result)

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [947]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
veri_tam = data[data["Ram (Sistem Belleği) Tipi"].notnull()].copy()
eksik_veri =data[data["Ram (Sistem Belleği) Tipi"].isnull()].copy()

# Kategorik ve sayısal sütunları belirleme
categorical_data = ["Brand", "Model"]
sayısal_data = ["Price"]

# Tüm verilerle LabelEncoder'ı eğit
encoders = {}
for col in categorical_data:
    birlesik_veri = pd.concat([veri_tam[col], eksik_veri[col]]).astype(str)
    lb = LabelEncoder()
    lb.fit(birlesik_veri)
    encoders[col] = lb

    # Encode işlemini uygula
    veri_tam.loc[:, col] = lb.transform(veri_tam[col].astype(str))
    eksik_veri.loc[:, col] = lb.transform(eksik_veri[col].astype(str))

# Hedef değişkeni encode et
ram_encoder = LabelEncoder()
veri_tam["Ram (Sistem Belleği) Tipi"] = ram_encoder.fit_transform(veri_tam["Ram (Sistem Belleği) Tipi"])

# Giriş ve çıkış değişkenlerini ayır
x = veri_tam[categorical_data + sayısal_data]
y = veri_tam["Ram (Sistem Belleği) Tipi"]

# Veriyi ölçeklendirme
scaler = StandardScaler()
x = scaler.fit_transform(x)

# Eğitim ve test setlerini ayır
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Modeli eğitme
model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

# Test seti üzerinde doğruluğu kontrol et
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy:.2f}")

# Eksik veriler için giriş değişkenlerini hazırlama
x_eksik = eksik_veri[categorical_data + sayısal_data]
x_eksik = scaler.transform(x_eksik)

# Eksik veriler için tahmin yapma
tahminler = model.predict(x_eksik)

eksik_veri["Tahmin Edilen Ram (Sistem Belleği) Tipi"] = ram_encoder.inverse_transform(tahminler)
for col in ["Brand", "Model"]:
    eksik_veri[col] = encoders[col].inverse_transform(eksik_veri[col].astype(int))
    veri_tam[col] = encoders[col].inverse_transform(veri_tam[col].astype(int))

eksik_veri["Orijinal Ram (Sistem Belleği) Tipi"] = eksik_veri["Ram (Sistem Belleği) Tipi"]
eksik_veri["Ram (Sistem Belleği) Tipi"] = eksik_veri["Tahmin Edilen Ram (Sistem Belleği) Tipi"]

# Tamamlanan veri setini birleştirme

# Tamamlanan veri setini birleştirme
tamamlanmis_veri = pd.concat([veri_tam, eksik_veri])

# Sonuçları yazdırma
print("\nTamamlanan Veri Seti:")
tamamlanmis_veri

Model Doğruluğu: 0.95

Tamamlanan Veri Seti:


,Brand,Model,Price,Ram (Sistem Belleği) Tipi,Tahmin Edilen Ram (Sistem Belleği) Tipi,Orijinal Ram (Sistem Belleği) Tipi
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8750.0,8,NaN,NaN
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13499.0,0,NaN,NaN
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18299.0,4,NaN,NaN
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11005.0,4,NaN,NaN
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15498.0,4,NaN,NaN
...,...,...,...,...,...,...
4981,Apple,Macbook Pro M2 Çip 8c Cpu 10c Gpu 16gb 512 Gb ...,69999.0,Birleşik Bellek,Birleşik Bellek,NaN
5106,LENOVO,NB V15 82YU00P0TX RYZEN3-7320U 8GB 256SSD O/B ...,17500.0,DDR4,DDR4,NaN
5210,LENOVO,Laptop,19999.0,DDR4,DDR4,NaN
5279,LENOVO,Ideapad 3 14ADA05 AMD Ryzen 3 3250U Laptop 4/2...,13599.0,DDR4,DDR4,NaN


In [1014]:
data=df.copy()

In [1236]:
def Ml_fillna(data,colon):
    
    veri_tam = data[data[colon].notnull()].copy()
    eksik_veri =data[data[colon].isnull()].copy()
    
    categorical_data = ["Brand", "Model"]
    sayısal_data = ["Price"]
    encoders = {}
    for col in categorical_data:
        birlesik_veri = pd.concat([veri_tam[col], eksik_veri[col]]).astype(str)
        lb = LabelEncoder()
        lb.fit(birlesik_veri)
        encoders[col] = lb
        
        veri_tam.loc[:, col] = lb.transform(veri_tam[col].astype(str))
        eksik_veri.loc[:, col] = lb.transform(eksik_veri[col].astype(str))
    
    ram_encoder = LabelEncoder()
    veri_tam[colon] = ram_encoder.fit_transform(veri_tam[colon])
    
    x = veri_tam[categorical_data + sayısal_data]
    y = veri_tam[colon]
    
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    models = [
          ('LG', LogisticRegression(solver='saga', max_iter=50)),
          ('KNN', KNeighborsClassifier()),
          ('CART', DecisionTreeClassifier()),
          ('HistgBM',HistGradientBoostingClassifier()),
          ('RF', RandomForestClassifier(random_state=42)),
          
          ("LightGBM", LGBMClassifier(verbose=-1))]
    
    result={'Model':[],'Acc':[],'Acc_Balanced':[]}
    for name, classifier in models:
        classifier.fit(x_train, y_train)
        y_pred=classifier.predict(x_test)
        acc=accuracy_score(y_test,y_pred)
        acc_balanced=balanced_accuracy_score(y_test,y_pred)
        result['Model'].append(name)
        result['Acc'].append(acc)
        result['Acc_Balanced'].append(acc_balanced)
    
    result=pd.DataFrame(result)
    
    print(result)
    
    best_model_name = result.loc[result['Acc'].idxmax(), 'Model']
    print(f"En Yüksek ACC Değerine Sahip Model: {best_model_name}")
    
    best_model = None
    for name, classifier in models:
        if name == best_model_name:
            best_model = classifier
            break
    
    x_eksik = eksik_veri[categorical_data + sayısal_data]
    x_eksik = scaler.transform(x_eksik)
    
    tahminler = best_model.predict(x_eksik)
    eksik_veri["Tahmin Edilen"] = ram_encoder.inverse_transform(tahminler)
    for col in ["Brand", "Model"]:
        eksik_veri[col] = encoders[col].inverse_transform(eksik_veri[col].astype(int))
        veri_tam[col] = encoders[col].inverse_transform(veri_tam[col].astype(int))
        
    eksik_veri["Orijinal"] = eksik_veri[colon]
    eksik_veri[colon] = eksik_veri["Tahmin Edilen"]
    
    tamamlanmis_veri = pd.concat([veri_tam, eksik_veri])
    
    for idx,row in eksik_veri.iterrows():
        data.loc[idx, colon] = row[colon]
    
    
    return eksik_veri[['Brand','Model',colon,"Orijinal","Tahmin Edilen"]]
    

In [1036]:
Ml_fillna(df,'Klavye')

       Model       Acc  Acc_Balanced
0         LG  0.526032      0.204526
1        KNN  0.720826      0.435250
2       CART  0.940754      0.852248
3    HistgBM  0.859066      0.649604
4         RF  0.951526      0.849319
5    XGBoost  0.877020      0.792626
6  XGBoostRF  0.673250      0.394498
7   LightGBM  0.851885      0.775394
En Yüksek ACC Değerine Sahip Model: RF


,Brand,Model,Klavye,Orijinal,Tahmin Edilen
5,Zeiron,Mirage X32 Pro White Ryzen 7 5700X 32GB RAM 1T...,RGB,RGB,RGB
17,Zeiron,Damage X50 Ryzen 7 5700X 32GB 1TB M.2 RTX4060T...,RGB,RGB,RGB
20,Zeiron,Mirage X32 Ryzen 7 5700X 32GB RAM 1TB M.2 Nvme...,RGB,RGB,RGB
29,Zeiron,Aurora Gs17 Ryzen 5 5500 16GB DDR4 256GB M.2 S...,RGB,RGB,RGB
32,Creature,CRTG102 Ryzen 5 5600 16GB DDR4 512GB NVMe RTX ...,RGB,RGB,RGB
44,Zeiron,Scorpion X70 Ryzen 7 5700X 32GB DDR4 RAM 1TB N...,RGB,RGB,RGB
53,Zeiron,Wizard Simple5 Ryzen 5 5600x 32gb 512gb M.2 Ss...,RGB,RGB,RGB
68,Zeiron,xTreme V2 Ryzen 7 5700X 32GB RAM 1TB M.2 Nvme ...,RGB,RGB,RGB
89,Zeiron,Prime X5 Ryzen 5 5500 16GB 3200MHz Ram 512GB N...,RGB,RGB,RGB
95,Zeiron,BS19 Intel i5 12400F 32GB RAM 1TB Nvme M.2 SSD...,RGB,RGB,RGB


In [1037]:
column_summary_table(df,'Klavye')

,Özellik,Değer
0,Sütun Adı,Klavye
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,7
4,Değer Sayımları,"{'Q Türkçe': 2538, 'RGB': 1640, 'RGB Numeric':..."
5,Dağılım - count,5616
6,Dağılım - unique,7
7,Dağılım - top,Q Türkçe
8,Dağılım - freq,2538


In [1051]:
column_summary_table(df,'Çözünürlük Standartı')

,Özellik,Değer
0,Sütun Adı,Çözünürlük Standartı
1,Veri Türü,object
2,NaN Değer Sayısı,40
3,Benzersiz Değer Sayısı,15
4,Değer Sayımları,"{'Full HD (FHD)': 4240, 'WUXGA': 226, 'FHD+': ..."
5,Dağılım - count,5576
6,Dağılım - unique,15
7,Dağılım - top,Full HD (FHD)
8,Dağılım - freq,4240


In [1049]:
df['Çözünürlük Standartı']=df['Çözünürlük Standartı'].apply(Belirtilmemis)

In [1053]:
Ml_fillna(df,'Çözünürlük Standartı')

       Model       Acc  Acc_Balanced
0         LG  0.741935      0.081904
1        KNN  0.815412      0.391036
2       CART  0.948029      0.840317
3    HistgBM  0.710573      0.086698
4         RF  0.955197      0.852499
5    XGBoost  0.935484      0.799921
6  XGBoostRF  0.794803      0.269338
7   LightGBM  0.496416      0.067782
En Yüksek ACC Değerine Sahip Model: RF


,Brand,Model,Çözünürlük Standartı,Orijinal,Tahmin Edilen
29,Zeiron,Aurora Gs17 Ryzen 5 5500 16GB DDR4 256GB M.2 S...,Full HD (FHD),NaN,Full HD (FHD)
32,Creature,CRTG102 Ryzen 5 5600 16GB DDR4 512GB NVMe RTX ...,Full HD (FHD),NaN,Full HD (FHD)
89,Zeiron,Prime X5 Ryzen 5 5500 16GB 3200MHz Ram 512GB N...,Full HD (FHD),NaN,Full HD (FHD)
95,Zeiron,BS19 Intel i5 12400F 32GB RAM 1TB Nvme M.2 SSD...,Full HD (FHD),NaN,Full HD (FHD)
101,Zeiron,Acx17 Ryzen 5 5500 16gb 512gb Nvme M.2 Ssd Nvi...,Full HD Ultra Wide,NaN,Full HD Ultra Wide
113,Zeiron,Npx55 I5-10400f 16gb 256gb M.2 Ssd 1tb Hdd Gtx...,Full HD (FHD),NaN,Full HD (FHD)
119,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,Full HD (FHD),NaN,Full HD (FHD)
137,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,Full HD (FHD),NaN,Full HD (FHD)
140,Zeiron,C-Navik X40 Ryzen 5 5600 32GB Ram 512GB NVMe M...,Full HD (FHD),NaN,Full HD (FHD)
143,Creature,16GB DDR4 512GB NVMe Radeon RX 7600 24 inç Mon...,Full HD (FHD),NaN,Full HD (FHD)


In [1080]:
df.isnull().sum().sort_values(ascending=False)

SSD Teknolojisi                5612
Ekran Özelliği                 5608
Görüntü Kalitesi               5608
Ekran Kartı Gücü               5280
Optik Sürücü Tipi              4016
Arttırılabilir Azami Bellek    3264
Temel İşlemci Hızı (GHz)       3186
Maksimum İşlemci Hızı (GHz)    3128
Ekran Yenileme Hızı             992
İşlemci Modeli                  560
Ekran Kartı Bellek Tipi         554
İşlemci Çekirdek Sayısı         208
İşlemci Nesli                    92
Renk                             82
Hard Disk Kapasitesi             30
Ekran Kartı Hafızası             16
Garanti Tipi                      0
Klavye                            0
Bağlantılar                       0
Panel Tipi                        0
Çözünürlük Standartı              0
Garanti Süresi                    0
HDMI                              0
Brand_Encoded                     0
Ram (Sistem Belleği) Tipi         0
Brand                             0
Dokunmatik Ekran                  0
Ram (Sistem Belleği)        

In [1060]:
Ml_fillna(df,'Çözünürlük')

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


       Model       Acc  Acc_Balanced
0         LG  0.777281      0.062930
1        KNN  0.847048      0.375392
2       CART  0.958855      0.928149
3    HistgBM  0.757603      0.047780
4         RF  0.964222      0.927604
5    XGBoost  0.952594      0.891720
6  XGBoostRF  0.819320      0.277628
7   LightGBM  0.581395      0.080746
En Yüksek ACC Değerine Sahip Model: RF


c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


,Brand,Model,Çözünürlük,Orijinal,Tahmin Edilen
29,Zeiron,Aurora Gs17 Ryzen 5 5500 16GB DDR4 256GB M.2 S...,1920 x 1080,NaN,1920 x 1080
32,Creature,CRTG102 Ryzen 5 5600 16GB DDR4 512GB NVMe RTX ...,1920 x 1200,NaN,1920 x 1200
44,Zeiron,Scorpion X70 Ryzen 7 5700X 32GB DDR4 RAM 1TB N...,1920 x 1080,NaN,1920 x 1080
86,Zeiron,Xtreme V60 Ryzen 7 5700x 32gb Ram 1tb M.2 Nvme...,1920 x 1080,NaN,1920 x 1080
95,Zeiron,BS19 Intel i5 12400F 32GB RAM 1TB Nvme M.2 SSD...,1920 x 1080,NaN,1920 x 1080
113,Zeiron,Npx55 I5-10400f 16gb 256gb M.2 Ssd 1tb Hdd Gtx...,1920 x 1080,NaN,1920 x 1080
119,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,1920 x 1080,NaN,1920 x 1080
122,Zeiron,Tsx99 Ryzen 5 3600 32GB DDR4 Ram 512GB SSD RX5...,1920 x 1080,NaN,1920 x 1080
137,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,1920 x 1080,NaN,1920 x 1080
140,Zeiron,C-Navik X40 Ryzen 5 5600 32GB Ram 512GB NVMe M...,1920 x 1080,NaN,1920 x 1080


In [1072]:
Ml_fillna(df,'Panel Tipi')

       Model       Acc  Acc_Balanced
0         LG  0.767677      0.132477
1        KNN  0.834596      0.425026
2       CART  0.957071      0.857596
3    HistgBM  0.948232      0.842812
4         RF  0.959596      0.857937
5    XGBoost  0.946970      0.834701
6  XGBoostRF  0.856061      0.303235
7   LightGBM  0.958333      0.846281
En Yüksek ACC Değerine Sahip Model: RF


,Brand,Model,Panel Tipi,Orijinal,Tahmin Edilen
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,TN,NaN,TN
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,TN,NaN,TN
5,Zeiron,Mirage X32 Pro White Ryzen 7 5700X 32GB RAM 1T...,VA,NaN,VA
7,LENOVO,IdeaPad1 Intel Celeron N4020 4GB 128GB SSD Win...,TN,NaN,TN
12,ASUS,Vivobook Go 15 Amd Ryzen 3 7320u 8gb 256gb Ssd...,LED,NaN,LED
...,...,...,...,...,...
5604,ACER,Intel I5-12450H 8gb 256GB SSD UHD Graphics 15....,IPS,NaN,IPS
5605,HP,"32GB 2tb SSD RTX2050-4GB WIN11HOME 14"" Fhd 8A5...",IPS,NaN,IPS
5608,ASUS,Vivobook 15 X1502ZA-EJ2135 Intel Core i5 8GB 5...,LED,NaN,LED
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,TN,NaN,TN


In [1078]:
Ml_fillna(df,'Bağlantılar')

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


       Model       Acc  Acc_Balanced
0         LG  0.569892      0.088727
1        KNN  0.593190      0.410269
2       CART  0.903226      0.884507
3    HistgBM  0.568100      0.073027
4         RF  0.922939      0.885160
5    XGBoost  0.818996      0.731259
6  XGBoostRF  0.623656      0.287354
7   LightGBM  0.844086      0.806944
En Yüksek ACC Değerine Sahip Model: RF


,Brand,Model,Bağlantılar,Orijinal,Tahmin Edilen
32,Creature,CRTG102 Ryzen 5 5600 16GB DDR4 512GB NVMe RTX ...,USB - Bluetooth - HDMI,NaN,USB - Bluetooth - HDMI
119,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,Bluetooth,NaN,Bluetooth
137,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,Bluetooth,NaN,Bluetooth
143,Creature,16GB DDR4 512GB NVMe Radeon RX 7600 24 inç Mon...,USB - Bluetooth - HDMI,NaN,USB - Bluetooth - HDMI
284,ASUS,Tuf Gaming A15 Fa507nu-lp051 6gb Rtx4050 Ryzen...,USB,NaN,USB
676,LENOVO,3050 ti i5 512 GB SSD 15.6'' 82K101EMTX,USB - Bluetooth - HDMI,NaN,USB - Bluetooth - HDMI
739,ASUS,Tuf Gamıng F15 Fx507zc4 I7-12700h 16gb 1tb Ssd...,Bluetooth - USB,NaN,Bluetooth - USB
970,Huawei,"5800h 16gb Ram 512gb Ssd 16"" 2.5k Fullview Win...",USB - Bluetooth - HDMI,NaN,USB - Bluetooth - HDMI
1065,ASUS,"12700h 16gb 1tb Ssd Rtx3050/4gb 144hz 15.6"" Fh...",USB - Bluetooth - HDMI,NaN,USB - Bluetooth - HDMI
1144,ASUS,"I7-12700 / 32GB / 1TB / 6GB RTX3060 / 15.6"" / ...",USB - Bluetooth - HDMI,NaN,USB - Bluetooth - HDMI


In [1065]:
df['İşlemci Modeli']=df['İşlemci Modeli'].apply(Belirtilmemis)

In [1301]:
nan_columns = df.columns[(df=="nan").any()].tolist()  # NaN içeren sütunlar
belirtilmemis_columns = df.columns[(df == "Belirtilmemiştir").any()].tolist()  # "belirtilmemiştir" içeren sütunlar

# Her iki durumu da içeren sütunlar
result_columns = set(nan_columns) | set(belirtilmemis_columns)

print("NaN içeren sütunlar:", nan_columns)
print('"belirtilmemiştir" içeren sütunlar:', belirtilmemis_columns)
print("Sonuç:", list(result_columns))

column_summary_table(df,nan_columns[0])

NaN içeren sütunlar: ['İşlemci Çekirdek Sayısı']
"belirtilmemiştir" içeren sütunlar: []
Sonuç: ['İşlemci Çekirdek Sayısı']


,Özellik,Değer
0,Sütun Adı,İşlemci Çekirdek Sayısı
1,Veri Türü,object
2,NaN Değer Sayısı,0
3,Benzersiz Değer Sayısı,12
4,Değer Sayımları,"{'8.0': 1406, '10.0': 1170, '6.0': 622, '14.0'..."
5,Dağılım - count,5616
6,Dağılım - unique,12
7,Dağılım - top,8.0
8,Dağılım - freq,1406


In [1308]:
data['İşlemci Çekirdek Sayısı']=data['İşlemci Çekirdek Sayısı'].apply(lambda x: str(x) if pd.notna(x) else x)

In [1312]:
df['İşlemci Çekirdek Sayısı']=data['İşlemci Çekirdek Sayısı']

In [1310]:
Ml_fillna(data,'İşlemci Çekirdek Sayısı')

      Model       Acc  Acc_Balanced
0        LG  0.373383      0.247216
1       KNN  0.654344      0.564603
2      CART  0.914972      0.904863
3   HistgBM  0.879852      0.853721
4        RF  0.933457      0.912190
5  LightGBM  0.886322      0.867425
En Yüksek ACC Değerine Sahip Model: RF


,Brand,Model,İşlemci Çekirdek Sayısı,Orijinal,Tahmin Edilen
27,LENOVO,Ryzen 5-5625U 16GB 512GB SSD 15.6 FHD FreeDos ...,6.0,NaN,6.0
49,Huawei,MateBook D 16 2024 13. Nesil Core i9 16GB+1TB ...,8.0,NaN,8.0
70,Huawei,Matebook 14 Intel Core Ultra 5 125H 16Gb Ram 5...,8.0,NaN,8.0
117,LENOVO,V14 82yt00jhtx R5-7520u 8gb 256gb Ssd 14'' Dos,4.0,NaN,4.0
150,LENOVO,IdeaPad 1 15IGL7 82V700HGTX N4020 8 GB 256 GB ...,2.0,NaN,2.0
...,...,...,...,...,...
5403,LENOVO,V15 G3 82tv009ctx Ryzen 7 5825u 16gb 512gb Ssd...,10.0,NaN,10.0
5418,Store,"i3-1215U 8GB 512GB 15.6"" Ubuntu",6.0,NaN,6.0
5430,LENOVO,256GB DOS,12.0,NaN,12.0
5528,Dell,"I7 1185g7 32gb 1tb Ssd Win 10 Pro 14"" Fhd Taş ...",14.0,NaN,14.0


In [1314]:
df['İşlemci Çekirdek Sayısı']=df['İşlemci Çekirdek Sayısı'].apply(lambda x: float(x))

In [1288]:
df['Arttırılabilir Azami Bellek']=df['Arttırılabilir Azami Bellek'].apply(lambda x: str(x) if pd.notna(x) else x)

In [1283]:
df['Temel İşlemci Hızı (GHz)']=df['Temel İşlemci Hızı (GHz)'].apply(lambda x: float(x))

In [1290]:
Ml_fillna(df,'Arttırılabilir Azami Bellek')

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


      Model       Acc  Acc_Balanced
0        LG  0.535032      0.181099
1       KNN  0.683652      0.431221
2      CART  0.932059      0.928552
3   HistgBM  0.927813      0.906705
4        RF  0.940552      0.900990
5  LightGBM  0.919321      0.890133
En Yüksek ACC Değerine Sahip Model: RF


,Brand,Model,Arttırılabilir Azami Bellek,Orijinal,Tahmin Edilen
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8.0,NaN,8.0
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,16.0,NaN,16.0
6,LENOVO,Ideapad AMD Ryzen 7 5700U 16GB 512GB SSD Freed...,16.0,NaN,16.0
7,LENOVO,IdeaPad1 Intel Celeron N4020 4GB 128GB SSD Win...,4.0,NaN,4.0
12,ASUS,Vivobook Go 15 Amd Ryzen 3 7320u 8gb 256gb Ssd...,8.0,NaN,8.0
...,...,...,...,...,...
5611,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",64.0,NaN,64.0
5612,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",64.0,NaN,64.0
5613,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",64.0,NaN,64.0
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,128.0,NaN,128.0


In [1262]:
df=df.drop('Optik Sürücü Tipi',axis=1)

In [1257]:
df=df.drop(['Ekran Kartı Tipi Encoded','Brand_Encoded','Görüntü Kalitesi','Ekran Özelliği','SSD Teknolojisi'],axis=1)

In [1246]:
df['Hard Disk Kapasitesi']=df['Hard Disk Kapasitesi'].apply(lambda x: str(x) if pd.notna(x) else x)

In [1251]:
df['Hard Disk Kapasitesi']=df['Hard Disk Kapasitesi'].apply(lambda x: float(x))

In [1248]:
Ml_fillna(df,'Hard Disk Kapasitesi')

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


      Model       Acc  Acc_Balanced
0        LG  0.915921      0.111111
1       KNN  0.908766      0.202943
2      CART  0.966011      0.809948
3   HistgBM  0.337209      0.138227
4        RF  0.978533      0.811467
5  LightGBM  0.587657      0.117407
En Yüksek ACC Değerine Sahip Model: RF


,Brand,Model,Hard Disk Kapasitesi,Orijinal,Tahmin Edilen
41,Zeiron,Amx60 Ryzen 5 3600 16GB 3200MHz 256GB SSD+1TB ...,0.0,NaN,0.0
74,Zeiron,Legend X80 Ryzen 5 3600 32gb 512gb M.2 Nvme Ss...,1.0,NaN,1.0
104,Zeiron,Sx75 i7-860 8GB 256GB SSD GT730 4GB Ekran Kart...,0.0,NaN,0.0
113,Zeiron,Npx55 I5-10400f 16gb 256gb M.2 Ssd 1tb Hdd Gtx...,0.0,NaN,0.0
179,LENOVO,IdeaPad Slim 3/ Intel N100/ 4GB Ram/ 128GB SSD...,0.0,NaN,0.0
227,LENOVO,Intel N100 4GB RAM 128Gb SSD Intel UHD Graphic...,0.0,NaN,0.0
336,HP,"5500U 8GB 256GB SSD Freedos 15.6"" FHD Taşınabi...",0.0,NaN,0.0
454,LENOVO,V14 82c2001htx Celeron N4020 4gb 128ssd O/b 14...,0.0,NaN,0.0
606,Gigabyte,9KG-43TRC54SD i7-13650HX 16GB 1TB SSD 8GB RTX ...,0.0,NaN,0.0
1302,MSI,A14vksg-481tr I9-14900hx 4tb Ssd 64gb Ddr5 12g...,0.0,NaN,0.0


In [1226]:
df.groupby(['Brand'])['Ekran Yenileme Hızı'].mean().reset_index()

,Brand,Ekran Yenileme Hızı
0,ACER,113.840000
1,ASUS,110.192857
2,Alienware,360.000000
3,Apple,114.888889
4,Arçelik,144.000000
5,CASPER,113.102041
6,Creature,144.000000
7,Dell,149.225806
8,EXA,60.000000
9,Everest,60.000000


In [1224]:
for i in x[x['Ekran Yenileme Hızı'].isna()]['Brand'].values:
    df.loc[(df['Brand']==i)&(df['Ekran Yenileme Hızı'].isna()),"Ekran Yenileme Hızı"]=144.0

In [1232]:
df['Ekran Yenileme Hızı']=df['Ekran Yenileme Hızı'].apply(lambda x: str(x) if pd.notna(x) else x )

In [1235]:
Ml_fillna(df,'Ekran Yenileme Hızı')

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


      Model       Acc  Acc_Balanced
0        LG  0.634015      0.231792
1       KNN  0.839612      0.719812
2      CART  0.956943      0.940677
3   HistgBM  0.369214      0.186925
4        RF  0.956943      0.923501
5  LightGBM  0.946179      0.897540
En Yüksek ACC Değerine Sahip Model: CART


,Brand,Model,Ekran Yenileme Hızı,Orijinal,Tahmin Edilen
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,90.0,NaN,90.0
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,60.0,NaN,60.0
5,Zeiron,Mirage X32 Pro White Ryzen 7 5700X 32GB RAM 1T...,60.0,NaN,60.0
15,Hometech,Alfa 620c Intel Celeron N4020 4 Gb 128 Gb Ssd ...,60.0,NaN,60.0
20,Zeiron,Mirage X32 Ryzen 7 5700X 32GB RAM 1TB M.2 Nvme...,60.0,NaN,60.0
...,...,...,...,...,...
5535,HP,"Ssd 15.6"" Fhd Windows 11 Pro Taşınabilir Bilgi...",60.0,NaN,60.0
5538,LENOVO,"V15 G3 Iap I5-1235U 40GB 512GB SSD 15.6"" WIN11...",60.0,NaN,60.0
5569,ASUS,"40gb 2tb Ssd 15.6"" Fhd W11p Bilgisayar B1502cb...",144.0,NaN,144.0
5605,HP,"32GB 2tb SSD RTX2050-4GB WIN11HOME 14"" Fhd 8A5...",144.0,NaN,144.0


In [1176]:
df[df['Ekran Kartı Gücü'].notnull()]['Brand'].value_counts()

Brand
MONSTER       142
ASUS           44
LENOVO         40
Game Garaj     34
MSI            32
ACER           30
HP              8
RAZER           6
Name: count, dtype: int64

In [1129]:
df['İşlemci Nesli']=df['İşlemci Nesli'].apply(Belirtilmemis)

In [1160]:
df['Maksimum İşlemci Hızı (GHz)']=df['Maksimum İşlemci Hızı (GHz)'].apply(lambda x:str(x))

In [1166]:
df['Maksimum İşlemci Hızı (GHz)']=data['Maksimum İşlemci Hızı (GHz)']

In [1180]:
Ml_fillna(df,'Renk')

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\sam

      Model       Acc  Acc_Balanced
0        LG  0.482385      0.027854
1       KNN  0.628726      0.218770
2      CART  0.911472      0.847462
3   HistgBM  0.432701      0.026286
4        RF  0.921409      0.854012
5  LightGBM  0.187895      0.022184
En Yüksek ACC Değerine Sahip Model: RF


c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


,Brand,Model,Renk,Orijinal,Tahmin Edilen
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,Siyah,NaN,Siyah
5,Zeiron,Mirage X32 Pro White Ryzen 7 5700X 32GB RAM 1T...,Siyah,NaN,Siyah
11,Zeiron,Prime X90 Ryzen 5 5500 32GB 1TB M.2 SSD RTX305...,Siyah,NaN,Siyah
17,Zeiron,Damage X50 Ryzen 7 5700X 32GB 1TB M.2 RTX4060T...,Siyah,NaN,Siyah
20,Zeiron,Mirage X32 Ryzen 7 5700X 32GB RAM 1TB M.2 Nvme...,Siyah,NaN,Siyah
...,...,...,...,...,...
2945,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,Siyah,NaN,Siyah
2948,Zeiron,C-Navik X40 Ryzen 5 5600 32GB Ram 512GB NVMe M...,Siyah,NaN,Siyah
2951,Creature,16GB DDR4 512GB NVMe Radeon RX 7600 24 inç Mon...,Siyah,NaN,Siyah
2954,Zeiron,Rzx80 Ryzen 5 3600 32GB Ram 1TB NVMe M.2 SSD R...,Siyah,NaN,Siyah


In [1107]:
Ml_fillna(df,'Ekran Kartı Hafızası')

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


       Model       Acc  Acc_Balanced
0         LG  0.614286      0.219899
1        KNN  0.798214      0.446318
2       CART  0.935714      0.882450
3    HistgBM  0.351786      0.168512
4         RF  0.958929      0.916296
5    XGBoost  0.913393      0.639653
6  XGBoostRF  0.774107      0.436846
7   LightGBM  0.918750      0.873233
En Yüksek ACC Değerine Sahip Model: RF


,Brand,Model,Ekran Kartı Hafızası,Orijinal,Tahmin Edilen
32,Creature,CRTG102 Ryzen 5 5600 16GB DDR4 512GB NVMe RTX ...,1.0,NaN,1.0
119,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,6.0,NaN,6.0
137,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,6.0,NaN,6.0
143,Creature,16GB DDR4 512GB NVMe Radeon RX 7600 24 inç Mon...,1.0,NaN,1.0
861,ASUS,UX5304MA-NQ154W Intel Ultra 7 155U 16GB RAM 1T...,1.0,NaN,1.0
1013,LENOVO,"82V700HGTX N4020, 8 GB DDR4 RAM, 256 GB SSD, 1...",4.0,NaN,4.0
2291,LENOVO,"8640HS/ 16GB Ram/ 512GB SSD/ 14""/ W11/ 83DK002VTR",6.0,NaN,6.0
2298,LENOVO,NB V15 82YU00P0TX RYZEN3-7320U 8GB 256SSD O/B ...,1.0,NaN,1.0
2840,Creature,CRTG102 Ryzen 5 5600 16GB DDR4 512GB NVMe RTX ...,1.0,NaN,1.0
2927,Creature,Crtg4 Ryzen 7 7700 16gb Ddr5 512gb Nvme Rtx 40...,6.0,NaN,6.0


In [1158]:
df['Ekran Yenileme Hızı']=df['Ekran Yenileme Hızı'].apply(lambda x:float(x))

In [1125]:
df['Hard Disk Kapasitesi']=df['Hard Disk Kapasitesi'].apply(lambda x:float(x))

In [1148]:
Ml_fillna(df,'İşlemci Modeli')

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\sam

      Model       Acc  Acc_Balanced
0        LG  0.258893      0.050345
1       KNN  0.559289      0.263771
2      CART  0.924901      0.911386
3   HistgBM  0.073123      0.012051
4        RF  0.942688      0.918641
5  LightGBM  0.065217      0.034386
En Yüksek ACC Değerine Sahip Model: RF


c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


,Brand,Model,İşlemci Modeli,Orijinal,Tahmin Edilen
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,1215U,NaN,1215U
5,Zeiron,Mirage X32 Pro White Ryzen 7 5700X 32GB RAM 1T...,12400F,NaN,12400F
11,Zeiron,Prime X90 Ryzen 5 5500 32GB 1TB M.2 SSD RTX305...,12400F,NaN,12400F
17,Zeiron,Damage X50 Ryzen 7 5700X 32GB 1TB M.2 RTX4060T...,10750H,NaN,10750H
20,Zeiron,Mirage X32 Ryzen 7 5700X 32GB RAM 1TB M.2 Nvme...,12400F,NaN,12400F
...,...,...,...,...,...
5521,HP,"8GB 512SSD MX550 15.6"" FullHD W11P Taşınabilir...",1355U,NaN,1355U
5568,Apple,"Apple M4 Pro 14?core CPU and 20?core GPU, 48GB...",M3,NaN,M3
5609,LENOVO,"Ryzen 7 Pro 5850u 24gb 4tb Ssd 14"" W11p 20xls3...",7840HS,NaN,7840HS
5610,LENOVO,"Ryzen 7 Pro 5850u 20gb 4tb Ssd 14"" W11p 20xls3...",7840HS,NaN,7840HS


In [1137]:
Ml_fillna(df,'Ekran Kartı Bellek Tipi')

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


      Model       Acc  Acc_Balanced
0        LG  0.614018      0.136938
1       KNN  0.772952      0.412434
2      CART  0.938796      0.878151
3   HistgBM  0.471866      0.140856
4        RF  0.950642      0.888496
5  LightGBM  0.937808      0.864542
En Yüksek ACC Değerine Sahip Model: RF


,Brand,Model,Ekran Kartı Bellek Tipi,Orijinal,Tahmin Edilen
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,Dahili,NaN,Dahili
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,Dahili,NaN,Dahili
15,Hometech,Alfa 620c Intel Celeron N4020 4 Gb 128 Gb Ssd ...,DDR,NaN,DDR
28,ACER,Aspire 3 A315-44p Ryzen 7 5700u 16 Gb 512 Gb S...,Dahili,NaN,Dahili
31,LENOVO,Ideapad 1 Intel Celeron N4020 8GB 256GB SSD DO...,DDR,NaN,DDR
...,...,...,...,...,...
5571,LENOVO,Ideapad 3 Amd Ryzen 3 3250u 8gb 256gb Ssd 15.6...,Dahili,NaN,Dahili
5575,CASPER,Ryzen 7 5700U-16Gb-500Gb Ssd-15.6inc-W11,Dahili,NaN,Dahili
5609,LENOVO,"Ryzen 7 Pro 5850u 24gb 4tb Ssd 14"" W11p 20xls3...",GDDR6,NaN,GDDR6
5610,LENOVO,"Ryzen 7 Pro 5850u 20gb 4tb Ssd 14"" W11p 20xls3...",GDDR6,NaN,GDDR6


In [965]:
df['Klavye']=df['Klavye'].apply(Belirtilmemis)

In [985]:
replace_dic={"RGB Türkçe":"RGB",
             "Q Türkçe (RGB Aydınlatmalı)":"RGB",
             "Q Türkçe + Numerik (RGB Aydınlatmalı)":"RGB",
             "Q Türkçe (Aydınlatmalı)":"RGB",
             "Q Türkçe + Numerik (RGB Aydınlatmalı)":"RGB Numeric",
             "Q Türkçe + Numerik (Aydınlatmalı)":"RGB Numeric",
             "Numerik Tuşlu":"Numeric",
             "Q Türkçe (Aydınlatmasız)":"Q Türkçe",
             "Q Türkçe + Numerik (Aydınlatmasız)":"Numeric",
             "Yok":"Belirtilmemiş"}

In [986]:
df['Klavye']=df['Klavye'].replace(replace_dic)

In [993]:
df['Klavye']=df['Klavye'].replace({"Belirtilmemis":"Belirtilmemiş"})

In [996]:
df['Klavye']=df['Klavye'].apply(Belirtilmemis)

Clear Data 

In [1318]:
df.isnull().sum()

Brand                          0
Model                          0
Price                          0
Rating                         0
Link                           0
Favori Number                  0
Rush Delivery                  0
Free Cargo                     0
İşlemci Tipi                   0
Ram (Sistem Belleği)           0
SSD Kapasitesi                 0
İşletim Sistemi                0
Ekran Kartı                    0
Ekran Boyutu                   0
Çözünürlük                     0
Kullanım Amacı                 0
Cihaz Ağırlığı                 0
Ekran Yenileme Hızı            0
Ekran Kartı Tipi               0
Ekran Kartı Hafızası           0
İşlemci Modeli                 0
İşlemci Nesli                  0
İşlemci Çekirdek Sayısı        0
Hard Disk Kapasitesi           0
Dokunmatik Ekran               0
Panel Tipi                     0
Ram (Sistem Belleği) Tipi      0
Renk                           0
Garanti Tipi                   0
Temel İşlemci Hızı (GHz)       0
Maksimum İ

In [1319]:
df.to_csv('Trendyol_Clear_Data_Computer_Price.csv', index=False, encoding='utf-8-sig')

In [68]:
df=pd.read_csv("Trendyol_Clear_Data_Computer_Price.csv")
df

,Brand,Model,Price,Rating,Link,Favori Number,Rush Delivery,Free Cargo,İşlemci Tipi,Ram (Sistem Belleği),...,Garanti Tipi,Temel İşlemci Hızı (GHz),Maksimum İşlemci Hızı (GHz),Bağlantılar,Klavye,Ekran Kartı Bellek Tipi,Çözünürlük Standartı,Garanti Süresi,Arttırılabilir Azami Bellek,HDMI
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8750.00,4.2,https://www.trendyol.com/casper/nirvana-c370-4...,60500.0,0,1,Intel Celeron,4.0,...,Casper Türkiye Garantili,1.1,2.8,Bluetooth,Q Türkçe,Dahili,HD Ready (HD),1.0,8.0,Yok
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13499.00,4.5,https://www.trendyol.com/lenovo/nb-v15-82yu012...,18500.0,1,1,AMD Ryzen 5,16.0,...,Lenovo Türkiye Garantili,1.1,2.8,Wi-Fi,F Klavye,Dahili,Full HD (FHD),2.0,32.0,Yok
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18299.00,4.6,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,30300.0,1,1,AMD Ryzen 5,32.0,...,Zeiron Türkiye Garantili,3.6,4.4,HDMI,RGB,DDR4,Full HD (FHD),2.0,64.0,Var
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11005.00,3.8,https://www.trendyol.com/lenovo/ideapad3-intel...,4354.0,1,1,Intel Core i3,8.0,...,Lenovo Türkiye Garantili,4.0,4.0,USB - Bluetooth - HDMI,Q Türkçe,Dahili,Full HD (FHD),2.0,16.0,Yok
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15498.00,4.5,https://www.trendyol.com/asus/vivobook-15-inte...,7974.0,1,1,Intel Core i5,8.0,...,Asus Türkiye Garantili,1.3,4.4,USB - Bluetooth - HDMI,Q Türkçe,DDR5,Full HD (FHD),2.0,8.0,Yok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43399.00,0.0,https://www.trendyol.com/lenovo/thinkpad-t14-g...,0.0,1,1,AMD Ryzen 7,48.0,...,Resmi Distribütör Garantili,1.9,4.4,Bluetooth - USB,Q Türkçe,Dahili,Full HD (FHD),3.0,64.0,Yok
5612,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",75716.13,0.0,https://www.trendyol.com/hp/victus-16-ryzen-7-...,0.0,0,1,AMD Ryzen 7,24.0,...,HP Türkiye Garantili,1.9,5.0,USB - Bluetooth - HDMI,RGB,GDDR6,Full HD (FHD),2.0,64.0,Yok
5613,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",39655.09,0.0,https://www.trendyol.com/hp/probook-450-g9-i5-...,0.0,0,1,Intel Core i5,64.0,...,HP Türkiye Garantili,1.7,4.4,USB - Bluetooth - HDMI,Q Türkçe,Dahili,Full HD (FHD),2.0,64.0,Yok
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,24464.30,5.0,https://www.trendyol.com/lenovo/v15-g3-intel-c...,0.0,0,1,Intel Core i7,24.0,...,Lenovo Türkiye Garantili,3.5,4.2,Bluetooth - HDMI,Q Türkçe,Dahili,Full HD (FHD),2.0,128.0,Yok


In [1338]:
dfa=df.copy()

In [1339]:
dfa["Hard Disk Kapasitesi"].value_counts()

Hard Disk Kapasitesi
0.000000    5090
0.500000     176
0.488281     112
1.000000     104
0.250000      78
0.125000      18
0.244141      16
2.000000      14
0.031250       4
6.000000       2
0.732422       2
Name: count, dtype: int64

In [1343]:
dfa["Hard Disk Kapasitesi"]=dfa["Hard Disk Kapasitesi"].apply(lambda x: str(x) if pd.notna(x) else x )

In [1346]:
Ml_fillna(dfa,"Hard Disk Kapasitesi")

c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\samet\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


      Model       Acc  Acc_Balanced
0        LG  0.481132      0.196875
1       KNN  0.632075      0.506228
2      CART  0.924528      0.957386
3   HistgBM  0.962264      0.976136
4        RF  0.924528      0.957386
5  LightGBM  0.915094      0.838636
En Yüksek ACC Değerine Sahip Model: HistgBM


,Brand,Model,Hard Disk Kapasitesi,Orijinal,Tahmin Edilen
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,0.48828125,NaN,0.48828125
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,1.0,NaN,1.0
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,0.125,NaN,0.125
5,Zeiron,Mirage X32 Pro White Ryzen 7 5700X 32GB RAM 1T...,1.0,NaN,1.0
6,LENOVO,Ideapad AMD Ryzen 7 5700U 16GB 512GB SSD Freed...,0.5,NaN,0.5
...,...,...,...,...,...
5611,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",1.0,NaN,1.0
5612,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",1.0,NaN,1.0
5613,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",1.0,NaN,1.0
5614,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,0.5,NaN,0.5


In [1348]:
dfa["Hard Disk Kapasitesi"].value_counts()

Hard Disk Kapasitesi
0.5            2052
1.0            1526
0.48828125     1140
0.25            652
2.0             100
0.125            72
0.244140625      58
0.732421875       8
0.03125           6
6.0               2
Name: count, dtype: int64

In [1349]:
dfa["Hard Disk Kapasitesi"].isnull().sum()

0

In [1341]:
dfa.loc[dfa["Hard Disk Kapasitesi"] == 0, "Hard Disk Kapasitesi"] = np.nan

Comments Analyzing

In [13]:
df_comment=pd.read_csv("Trendyol_Computer_Comments.csv")
df_comment

,Original Link,Comment Link,Yorumlar
0,https://www.trendyol.com/casper/nirvana-c370-4...,https://www.trendyol.com/casper/nirvana-c370-4...,"[""daha çok yeni olduğu için performansına dair..."
1,https://www.trendyol.com/lenovo/nb-v15-82yu012...,https://www.trendyol.com/lenovo/nb-v15-82yu012...,['çok begendim ikincisini sipariş verecem tavs...
2,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,['Oğlum için aldım harika bir ürün tavsiye ede...
3,https://www.trendyol.com/lenovo/ideapad3-intel...,https://www.trendyol.com/lenovo/ideapad3-intel...,['sorunsuz geldi teşekkür ederim satıcıya karn...
4,https://www.trendyol.com/asus/vivobook-15-inte...,https://www.trendyol.com/asus/vivobook-15-inte...,"['Kızıma aldım çok memnun', '12 yaşındaki orta..."
...,...,...,...
5611,https://www.trendyol.com/lenovo/thinkpad-t14-g...,https://www.trendyol.com/lenovo/thinkpad-t14-g...,['Yorum Yoktur']
5612,https://www.trendyol.com/hp/victus-16-ryzen-7-...,https://www.trendyol.com/hp/victus-16-ryzen-7-...,['Yorum Yoktur']
5613,https://www.trendyol.com/hp/probook-450-g9-i5-...,https://www.trendyol.com/hp/probook-450-g9-i5-...,['Yorum Yoktur']
5614,https://www.trendyol.com/lenovo/v15-g3-intel-c...,https://www.trendyol.com/lenovo/v15-g3-intel-c...,['Yorum Yoktur']


In [7]:
df_comment["Original Link"].value_counts()

Original Link
https://www.trendyol.com/zeiron/fx50-i5-3470-16gb-ram-1tb-hdd-256gb-ssd-gt730-4gb-23-8-oyuncu-masaustu-bilgisayar-p-50439438?boutiqueId=61&merchantId=127037                     4
https://www.trendyol.com/msi/katana-17-b13vfk37-i7-13620h-64gb-2tb-ssd-rtx4060-8gb-105w-17-3-fhd-144hz-w11h-per4-canta-p-867533460?boutiqueId=61&merchantId=668896               4
https://www.trendyol.com/creature/crtg4-ryzen-7-7700-16gb-ddr5-512gb-nvme-rtx-4060ti-gaming-hazir-sistem-p-850436214?boutiqueId=61&merchantId=827203                             4
https://www.trendyol.com/lenovo/ideapad-duet-5-12iru8-i7-1355u-16gb-512gb-ssd-12-4-2-5k-freedos-tasinabilir-bilgisayar-83b30048tx-p-856341097?boutiqueId=61&merchantId=104683    4
https://www.trendyol.com/hp/250-g10-968l4ete3-i7-1355u-16gb-512gb-ssd-15-6-fhd-freedos-p-873022361?boutiqueId=61&merchantId=109239                                               4
                                                                                           

In [14]:
df_new=df_comment.drop_duplicates(subset='Original Link')

In [15]:
df_new

,Original Link,Comment Link,Yorumlar
0,https://www.trendyol.com/casper/nirvana-c370-4...,https://www.trendyol.com/casper/nirvana-c370-4...,"[""daha çok yeni olduğu için performansına dair..."
1,https://www.trendyol.com/lenovo/nb-v15-82yu012...,https://www.trendyol.com/lenovo/nb-v15-82yu012...,['çok begendim ikincisini sipariş verecem tavs...
2,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,['Oğlum için aldım harika bir ürün tavsiye ede...
3,https://www.trendyol.com/lenovo/ideapad3-intel...,https://www.trendyol.com/lenovo/ideapad3-intel...,['sorunsuz geldi teşekkür ederim satıcıya karn...
4,https://www.trendyol.com/asus/vivobook-15-inte...,https://www.trendyol.com/asus/vivobook-15-inte...,"['Kızıma aldım çok memnun', '12 yaşındaki orta..."
...,...,...,...
2803,https://www.trendyol.com/lenovo/thinkpad-t14-g...,https://www.trendyol.com/lenovo/thinkpad-t14-g...,['Yorum Yoktur']
2804,https://www.trendyol.com/hp/victus-16-ryzen-7-...,https://www.trendyol.com/hp/victus-16-ryzen-7-...,['Yorum Yoktur']
2805,https://www.trendyol.com/hp/probook-450-g9-i5-...,https://www.trendyol.com/hp/probook-450-g9-i5-...,['Yorum Yoktur']
2806,https://www.trendyol.com/lenovo/v15-g3-intel-c...,https://www.trendyol.com/lenovo/v15-g3-intel-c...,['Yorum Yoktur']


In [69]:
df=df.drop_duplicates(subset='Link')

In [21]:
for col in ["Brand","Model"]:
    df_new[col]=df[col]
df_new.columns=["Original Link","Comment Link","Yorumlar","Brand","Model"]

C:\Users\samet\AppData\Local\Temp\ipykernel_5440\1227371042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[col]=df[col]
C:\Users\samet\AppData\Local\Temp\ipykernel_5440\1227371042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[col]=df[col]


In [24]:
df_new["Yorumlar"]

0       ["daha çok yeni olduğu için performansına dair...
1       ['çok begendim ikincisini sipariş verecem tavs...
2       ['Oğlum için aldım harika bir ürün tavsiye ede...
3       ['sorunsuz geldi teşekkür ederim satıcıya karn...
4       ['Kızıma aldım çok memnun', '12 yaşındaki orta...
                              ...                        
2803                                     ['Yorum Yoktur']
2804                                     ['Yorum Yoktur']
2805                                     ['Yorum Yoktur']
2806                                     ['Yorum Yoktur']
2807                                     ['Yorum Yoktur']
Name: Yorumlar, Length: 2793, dtype: object

In [48]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# VADER duygu analizi aracı
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(yorum_listesi):
    """Bir ürün için yorum listesini analiz edip pozitif, negatif, nötr oranlarını döndürür."""
    if isinstance(yorum_listesi, list):  # Yorumların listede olup olmadığını kontrol et
        # Listeyi tek bir string haline getir (Tüm yorumları birleştir)
        combined_text = " ".join(yorum_listesi)
        
        # Duygu analizi yap
        skor = analyzer.polarity_scores(combined_text)
        
        # Pozitif, negatif, nötr yüzdelerini al
        return skor["pos"], skor["neg"], skor["neu"]
    else:
        return None, None, None 

In [18]:
import ast
yorum=df_new["Yorumlar"].values[0]
yorumlar=ast.literal_eval(yorum)

In [65]:
df_new=df_new.drop("Commentss",axis=1)

In [19]:
df_new["Comments"] = df_new["Yorumlar"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

C:\Users\samet\AppData\Local\Temp\ipykernel_34540\29860784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Comments"] = df_new["Yorumlar"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [20]:
df_new

,Original Link,Comment Link,Yorumlar,Comments
0,https://www.trendyol.com/casper/nirvana-c370-4...,https://www.trendyol.com/casper/nirvana-c370-4...,"[""daha çok yeni olduğu için performansına dair...",[daha çok yeni olduğu için performansına dair ...
1,https://www.trendyol.com/lenovo/nb-v15-82yu012...,https://www.trendyol.com/lenovo/nb-v15-82yu012...,['çok begendim ikincisini sipariş verecem tavs...,[çok begendim ikincisini sipariş verecem tavsi...
2,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,https://www.trendyol.com/zeiron/zgx25-ryzen-5-...,['Oğlum için aldım harika bir ürün tavsiye ede...,[Oğlum için aldım harika bir ürün tavsiye eder...
3,https://www.trendyol.com/lenovo/ideapad3-intel...,https://www.trendyol.com/lenovo/ideapad3-intel...,['sorunsuz geldi teşekkür ederim satıcıya karn...,[sorunsuz geldi teşekkür ederim satıcıya karne...
4,https://www.trendyol.com/asus/vivobook-15-inte...,https://www.trendyol.com/asus/vivobook-15-inte...,"['Kızıma aldım çok memnun', '12 yaşındaki orta...","[Kızıma aldım çok memnun, 12 yaşındaki ortaoku..."
...,...,...,...,...
2803,https://www.trendyol.com/lenovo/thinkpad-t14-g...,https://www.trendyol.com/lenovo/thinkpad-t14-g...,['Yorum Yoktur'],[Yorum Yoktur]
2804,https://www.trendyol.com/hp/victus-16-ryzen-7-...,https://www.trendyol.com/hp/victus-16-ryzen-7-...,['Yorum Yoktur'],[Yorum Yoktur]
2805,https://www.trendyol.com/hp/probook-450-g9-i5-...,https://www.trendyol.com/hp/probook-450-g9-i5-...,['Yorum Yoktur'],[Yorum Yoktur]
2806,https://www.trendyol.com/lenovo/v15-g3-intel-c...,https://www.trendyol.com/lenovo/v15-g3-intel-c...,['Yorum Yoktur'],[Yorum Yoktur]


In [128]:
stoplist=stopwords.words('turkish')
stoplist

['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'çok',
 'çünkü',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'eğer',
 'en',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'için',
 'ile',
 'ise',
 'kez',
 'ki',
 'kim',
 'mı',
 'mu',
 'mü',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niçin',
 'niye',
 'o',
 'sanki',
 'şey',
 'siz',
 'şu',
 'tüm',
 've',
 'veya',
 'ya',
 'yani']

In [134]:
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r"([-?.!,/\"])", '', text)  # Temel noktalama temizliği
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,']", "", text)
    text = re.sub(r"[ ]+", " ", text)  # Fazla boşlukları temizle
    text = re.sub('\n\n', '', text)  # Gereksiz yeni satırları temizle
    text = text.rstrip().strip()

    
    words = text.split()
    cleaned_words = [word for word in words if word not in stoplist]

    return " ".join(cleaned_words)

In [163]:
def cleand_text(text):
    text = text.lower().strip()
    text = re.sub(r"([-?.!,/\"])", '', text)  # Temel noktalama temizliği
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,']", "", text)
    text = re.sub(r"[ ]+", " ", text)  # Fazla boşlukları temizle
    text = re.sub('\n\n', '', text)  # Gereksiz yeni satırları temizle
    text = text.rstrip().strip()
    return text

In [71]:
df_new["Brand"]=df["Brand"]
df_new["Model"]=df["Model"]
df_new["Rating"]=df["Rating"]
df_new["Favori Number"]=df["Favori Number"]
df_new["Price"]=df["Price"]

C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4171377806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Brand"]=df["Brand"]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4171377806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Model"]=df["Model"]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4171377806.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [254]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from googletrans import Translator

# Çevirici ve analizör başlat
translator = Translator()
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    # Metni İngilizce'ye çevir
    translated_text = translator.translate(text, src="tr", dest="en").text
    
    # Sentiment analizi yap
    score = analyzer.polarity_scores(translated_text)
    
    # Compound değerine göre sınıflandır
    if score['compound'] >= 0.05:
        sentiment = "Pozitif"
    elif score['compound'] <= -0.05:
        sentiment = "Negatif"
    else:
        sentiment = "Nötr"
    
    return sentiment, score

# Test metni
text = "Yeni Leptop aldım ancak leptop bozuk çıktı"
sentiment, score = get_sentiment(text)

print(f"Metin: {text}")
print(f"Çevrilen Metin: {translator.translate(text, src='tr', dest='en').text}")
print(f"Polarity Skorları: {score}")
print(f"Sonuç: {sentiment}")

Metin: Yeni Leptop aldım ancak leptop bozuk çıktı
Çevrilen Metin: I bought a new laptop but the laptop is broken
Polarity Skorları: {'neg': 0.316, 'neu': 0.684, 'pos': 0.0, 'compound': -0.631}
Sonuç: Negatif


In [182]:
result={"Text":[],"Translated Text":[],"Negatif":[],"Notr":[],"Positif":[],"Compound":[],"Sentiment":[]}
cleaned_texts = [cleand_text(yorum) for yorum in df_new["Comments"][871]]
tek_metin = " ".join(cleaned_texts)
sentiment, score = get_sentiment(tek_metin)
print(f"Metin: {tek_metin}")
print(f"Çevrilen Metin: {translator.translate(tek_metin, src='tr', dest='en').text}")
print(f"Polarity Skorları: {score}")
print(f"Sonuç: {sentiment}")
result["Text"].append(tek_metin)
result["Translated Text"].append(translator.translate(tek_metin, src='tr', dest='en').text)
for key,value in score.items():
    if key=="neg":
        result["Negatif"].append(value)
    elif key=="neu":
        result["Notr"].append(value)
    elif key=="pos":
        result["Positif"].append(value)
    else:
        result["Compound"].append(value)
result["Sentiment"].append(sentiment)

res=pd.DataFrame(result)
res


Metin: severek kullanıyorum gerçekten klavyesi inceliği dış görünüşü kısaca çok hoş kamerası klavyede bunu bilerek alın derim ben çok kullanmıyorum diye sorun etmiyorum alalı 1 yıldan fazla oldu sorun şimdilik yok hızlı şarj alması da harika gayet basarili tavsiye ediyorumsatici da cok ilgili tüm özellikleriyle muhteşem muhteşem harikası bir laptop aldığın para helal huvai çok memnun kaldık oğlum üniversitede kullaniyo 10 numara 5 yıldız ürün kaliteli ben cok beğenerek kullanıyorum tavsiye ederim herkese çok korkmuştum ilk yüklü bir internet alışverişimdi 😍çok teşekkür ederim elime sorunsuz bir şekilde ulaştı 🥰 özellikleri gerçekten çok iyi indirimde yakaladım 🙏 ürün gayet güzel geldi ofis kullanımı için gayet yeterli 2 aydır kullanıyorum her alanda mükemmel bir pc ve çokta şık ve kibar tam bir fiyat performans ürünü çok beğendim 16 inç ekran çok iyigüçlü bir işlemcisi var çok hızlı bir teslimat agt kuryeye teşekkürler kızımın okulu için aldım çok menmun fiyat perforsman ürünü i̇ndirim

,Text,Translated Text,Negatif,Notr,Positif,Compound,Sentiment
0,severek kullanıyorum gerçekten klavyesi inceli...,I use it fondly I really do not use it on the ...,0.026,0.693,0.281,0.9954,Pozitif


In [90]:
data={"Brand":[],"Model":[],"Price":[],"Favori Number":[],"Rating":[],"Text":[],"Translated Text":[],"Negatif":[],"Notr":[],"Positif":[],"Compound":[],"Sentiment":[]}

for i in range(len(df_new["Comments"][:100])):
    data["Brand"].append(df_new["Brand"][i])
    data["Model"].append(df_new["Model"][i])
    data["Price"].append(df_new["Price"][i])
    data["Favori Number"].append(df_new["Favori Number"][i])
    data["Rating"].append(df_new["Rating"][i])
    
    cleaned_texts = [clean_text(yorum) for yorum in df_new["Comments"][i]]
    tek_metin = " ".join(cleaned_texts)
    sentiment, score = get_sentiment(tek_metin)
        
        
    translated_text = translator.translate(tek_metin, src='tr', dest='en').text

        
    data["Text"].append(tek_metin)
    data["Translated Text"].append(translated_text)
    data["Negatif"].append(score["neg"])
    data["Notr"].append(score["neu"])
    data["Positif"].append(score["pos"])
    data["Compound"].append(score["compound"])
    data["Sentiment"].append(sentiment)

df_main=pd.DataFrame(data)
df_main 

,Brand,Model,Price,Favori Number,Rating,Text,Translated Text,Negatif,Notr,Positif,Compound,Sentiment
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8750.00,60500.0,4.2,daha çok yeni olduğu için performansına dair y...,I can not comment on the performance of more n...,0.014,0.690,0.296,0.9988,Pozitif
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13499.00,18500.0,4.5,çok begendim ikincisini sipariş verecem tavsiy...,I would like to order the latter.I have resear...,0.003,0.740,0.257,0.9993,Pozitif
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18299.00,30300.0,4.6,oğlum için aldım harika bir ürün tavsiye ederi...,I bought a great product for my son I recommen...,0.010,0.686,0.304,0.9996,Pozitif
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11005.00,4354.0,3.8,sorunsuz geldi teşekkür ederim satıcıya karne ...,Thank you. I bought it as a report card gift t...,0.040,0.728,0.232,0.9964,Pozitif
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15498.00,7974.0,4.5,kızıma aldım çok memnun 12 yaşındaki ortaokula...,I bought my daughter very satisfied 12 -year -...,0.045,0.645,0.310,0.9993,Pozitif
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Zeiron,Corvus X5 Ryzen 5 5600 16GB Ram 512GB NVMe M.2...,24999.00,1040.0,4.3,ürün çok güzel çocuklarım bayıldı hızlı gönder...,Product very nice children fainted in the fast...,0.043,0.725,0.232,0.9948,Pozitif
96,HP,8GB.RAM 512GB.SSD FDOS (Model:9G1G1ET),10776.90,112.0,0.0,açılımı çok hızlı fiyatına göre çok iyi bir le...,Meaning is a very good laptop for very fast price,0.000,0.738,0.262,0.4927,Pozitif
97,CASPER,Excalibur G870.1245-DFB0X-B i5-12450H 32GB RAM...,39417.71,13100.0,4.5,bir aydır kullanıyor oğlum gayet memnun teşekk...,"He has been using it for a month, my son is ve...",0.049,0.625,0.326,0.9974,Pozitif
98,Zeiron,Acx17 Ryzen 5 5500 16gb 512gb Nvme M.2 Ssd Nvi...,15999.00,352.0,4.6,sağlam bir şekilde 2 günde elime ulaştı tam ol...,I have reached my hand in 2 days in a firm way...,0.052,0.672,0.275,0.9992,Pozitif


In [124]:
len(df_main["Text"][0])

2482

In [126]:
len(df_main["Translated Text"][0])

1610

In [106]:
data = {
    "Brand": [],
    "Model": [],
    "Price": [],
    "Favori Number": [],
    "Rating": [],
    "Text": [],
    "Translated Text": [],
    "Negatif": [],
    "Notr": [],
    "Positif": [],
    "Compound": [],
    "Sentiment": []
}

for i in range(len(df_new["Comments"])):
    try:
        
        data["Brand"].append(df_new["Brand"][i])
        data["Model"].append(df_new["Model"][i])
        data["Price"].append(df_new["Price"][i])
        data["Favori Number"].append(df_new["Favori Number"][i])
        data["Rating"].append(df_new["Rating"][i])
        
       
        cleaned_texts = [clean_text(yorum) for yorum in df_new["Comments"][i]]
        tek_metin = " ".join(cleaned_texts)
        sentiment, score = get_sentiment(tek_metin)
        
        
        translated_text = translator.translate(tek_metin, src='tr', dest='en').text

        
        data["Text"].append(tek_metin)
        data["Translated Text"].append(translated_text)
        data["Negatif"].append(score["neg"])
        data["Notr"].append(score["neu"])
        data["Positif"].append(score["pos"])
        data["Compound"].append(score["compound"])
        data["Sentiment"].append(sentiment)
    
    except Exception as e:
        print(f"Hata veren indeks: {i}, Hata: {e}")  


df_main = pd.DataFrame(data)

df_main


Hata veren indeks: 122, Hata: the JSON object must be str, bytes or bytearray, not NoneType
Hata veren indeks: 364, Hata: 'NoneType' object is not iterable
Hata veren indeks: 727, Hata: 'NoneType' object is not iterable
Hata veren indeks: 933, Hata: Invalid input ConnectionInputs.RECV_PING in state ConnectionState.CLOSED
Hata veren indeks: 1106, Hata: the JSON object must be str, bytes or bytearray, not NoneType
Hata veren indeks: 1277, Hata: 'Translator' object has no attribute 'raise_Exception'
Hata veren indeks: 1459, Hata: 'Translator' object has no attribute 'raise_Exception'
Hata veren indeks: 1460, Hata: 'Translator' object has no attribute 'raise_Exception'
Hata veren indeks: 1472, Hata: 'Translator' object has no attribute 'raise_Exception'
Hata veren indeks: 1473, Hata: 'Translator' object has no attribute 'raise_Exception'
Hata veren indeks: 1494, Hata: 'Translator' object has no attribute 'raise_Exception'
Hata veren indeks: 1594, Hata: 'Translator' object has no attribute 

ValueError: All arrays must be of the same length

In [220]:
import pandas as pd
import numpy as np
from googletrans import Translator

# Google Translate başlat
translator = Translator()

# Yeni veri yapısı
data = {
    "Brand": [],
    "Model": [],
    "Price": [],
    "Favori Number": [],
    "Rating": [],
    "Text": [],
    "Translated Text": [],
    "Negatif": [],
    "Notr": [],
    "Positif": [],
    "Compound": [],
    "Sentiment": []
}


for i in range(1000,2794):
    try:
        # Temel bilgileri ekleyelim
        data["Brand"].append(df_new["Brand"][i])
        data["Model"].append(df_new["Model"][i])
        data["Price"].append(df_new["Price"][i])
        data["Favori Number"].append(df_new["Favori Number"][i])
        data["Rating"].append(df_new["Rating"][i])

        # 1️⃣ **Metni temizle**
        try:
            cleaned_texts = [cleand_text(yorum) for yorum in df_new["Comments"][i]]
            tek_metin = " ".join(cleaned_texts)
        except Exception as e:
            print(f"Temizleme hatası, indeks: {i}, Hata: {e}")
            tek_metin = np.nan  # Hata olursa NaN ekle

        data["Text"].append(tek_metin)

        # 2️⃣ **Duygu Analizi Yap**
        if isinstance(tek_metin, str) and tek_metin.strip():
            try:
                sentiment, score = get_sentiment(tek_metin)
                data["Negatif"].append(score["neg"])
                data["Notr"].append(score["neu"])
                data["Positif"].append(score["pos"])
                data["Compound"].append(score["compound"])
                data["Sentiment"].append(sentiment)
            except Exception as e:
                print(f"Duygu analizi hatası, indeks: {i}, Hata: {e}")
                data["Negatif"].append(np.nan)
                data["Notr"].append(np.nan)
                data["Positif"].append(np.nan)
                data["Compound"].append(np.nan)
                data["Sentiment"].append(np.nan)
        else:
            data["Negatif"].append(np.nan)
            data["Notr"].append(np.nan)
            data["Positif"].append(np.nan)
            data["Compound"].append(np.nan)
            data["Sentiment"].append(np.nan)

        # 3️⃣ **Çeviri Yap**
        if isinstance(tek_metin, str) and tek_metin.strip():
            try:
                translated_text = translator.translate(tek_metin, src='tr', dest='en').text
            except Exception as e:
                print(f"Çeviri hatası, indeks: {i}, Hata: {e}")
                translated_text = np.nan  # Hata olursa NaN ekle
        else:
            translated_text = np.nan

        data["Translated Text"].append(translated_text)

    except Exception as e:
        print(f"Genel hata, indeks: {i}, Hata: {e}")

# Pandas DataFrame'e çevir
df_mainx = pd.DataFrame(data)

# Sonuçları göster
df_mainx


Duygu analizi hatası, indeks: 1106, Hata: the JSON object must be str, bytes or bytearray, not NoneType
Çeviri hatası, indeks: 1106, Hata: the JSON object must be str, bytes or bytearray, not NoneType
Çeviri hatası, indeks: 1137, Hata: 'Translator' object has no attribute 'raise_Exception'
Çeviri hatası, indeks: 1144, Hata: 'Translator' object has no attribute 'raise_Exception'
Duygu analizi hatası, indeks: 1145, Hata: 'Translator' object has no attribute 'raise_Exception'
Çeviri hatası, indeks: 1146, Hata: 'Translator' object has no attribute 'raise_Exception'
Duygu analizi hatası, indeks: 1147, Hata: 'Translator' object has no attribute 'raise_Exception'
Duygu analizi hatası, indeks: 1157, Hata: 'Translator' object has no attribute 'raise_Exception'
Çeviri hatası, indeks: 1159, Hata: 'Translator' object has no attribute 'raise_Exception'
Duygu analizi hatası, indeks: 1162, Hata: 'Translator' object has no attribute 'raise_Exception'
Duygu analizi hatası, indeks: 1165, Hata: 'Translat

,Brand,Model,Price,Favori Number,Rating,Text,Translated Text,Negatif,Notr,Positif,Compound,Sentiment
0,CASPER,G870.1270-EX70R-B i7-12700H 64GB RAM 2TB NVMe ...,65299.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1,LENOVO,"IdeaPad 1 82V700A9TX01 N4020 4GB 256SSD 15.6"" ...",9399.00,1673.0,3.6,güzel görünüyor şarjı 10 saniye felan suruyo a...,"It looks beautiful, the charging is 10 seconds...",0.000,0.862,0.138,0.7155,Pozitif
2,Apple,"Macbook Air M3 16gb 256gb Ssd Macos 13"" Taşına...",44999.00,431.0,5.0,bi sorun olmadı şimdilik şimdi kurulum yapacağ...,"There was no problem for now, now I will set u...",0.000,0.746,0.254,0.6269,Pozitif
3,ASUS,G713QR-K4178 AMD RYZEN 7 5800H 16GB 1TB SSD RT...,81449.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
4,Apple,Apple M4 Max chip with 14?core CPU and 32?core...,157949.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
...,...,...,...,...,...,...,...,...,...,...,...,...
1788,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43399.00,0.0,0.0,yorum yoktur,NaN,NaN,NaN,NaN,NaN,NaN
1789,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",75716.13,0.0,0.0,yorum yoktur,NO COMMENT,NaN,NaN,NaN,NaN,NaN
1790,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",39655.09,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1791,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,24464.30,0.0,5.0,yorum yoktur,NaN,NaN,NaN,NaN,NaN,NaN


In [228]:
df_mainx.loc[(df_mainx["Text"]=="yorum yoktur")&(df_mainx["Translated Text"].isna())]

,Brand,Model,Price,Favori Number,Rating,Text,Translated Text,Negatif,Notr,Positif,Compound,Sentiment
146,GPTURKGRUP,3510 N8803VN3510_U I5-1135G7 8GB 256SSD O/B 15...,13888.00,0.0,0.0,yorum yoktur,NaN,0.688,0.312,0.0,-0.296,Negatif
159,ASUS,FA506NF-HN050A4 Ryzen5 7535HS 16GB 1TBSSD RTX2...,30409.00,0.0,0.0,yorum yoktur,NaN,0.688,0.312,0.0,-0.296,Negatif
169,HP,"I?5 12450h 32gb 256gb Ssd Rtx3050 Fdos 15.6"" F...",32649.00,0.0,0.0,yorum yoktur,NaN,0.688,0.312,0.0,-0.296,Negatif
173,LENOVO,"I7-13620H 16 GB 1 TBSSD 14"" Wuxga (1920X1200) ...",39782.61,0.0,0.0,yorum yoktur,NaN,0.688,0.312,0.0,-0.296,Negatif
180,MSI,"i5 12450H 8GB 512GB SSD RTX2050 W11P 15.6"" FHD...",28999.00,0.0,0.0,yorum yoktur,NaN,0.688,0.312,0.0,-0.296,Negatif
...,...,...,...,...,...,...,...,...,...,...,...,...
1786,LENOVO,"Ryzen 7 Pro 5850u 24gb 4tb Ssd 14"" W11p 20xls3...",46899.00,0.0,0.0,yorum yoktur,NaN,0.688,0.312,0.0,-0.296,Negatif
1787,LENOVO,"Ryzen 7 Pro 5850u 20gb 4tb Ssd 14"" W11p 20xls3...",46599.00,0.0,0.0,yorum yoktur,NaN,NaN,NaN,NaN,NaN,NaN
1788,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43399.00,0.0,0.0,yorum yoktur,NaN,NaN,NaN,NaN,NaN,NaN
1791,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,24464.30,0.0,5.0,yorum yoktur,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
result={"Text":[],"Translated Text":[],"Negatif":[],"Notr":[],"Positif":[],"Compound":[],"Sentiment":[]}
cleaned_texts = [clean_text(yorum) for yorum in df_new["Comments"][875]]
tek_metin = " ".join(cleaned_texts)
sentiment, score = get_sentiment(tek_metin)
print(f"Metin: {tek_metin}")
print(f"Çevrilen Metin: {translator.translate(tek_metin, src='tr', dest='en').text}")
print(f"Polarity Skorları: {score}")
print(f"Sonuç: {sentiment}")
result["Text"].append(tek_metin)
result["Translated Text"].append(translator.translate(tek_metin, src='tr', dest='en').text)
for key,value in score.items():
    if key=="neg":
        result["Negatif"].append(value)
    elif key=="neu":
        result["Notr"].append(value)
    elif key=="pos":
        result["Positif"].append(value)
    else:
        result["Compound"].append(value)
result["Sentiment"].append(sentiment)

res=pd.DataFrame(result)
res


Metin: gayet güzel eşime hediye aldım beğendi teşekkürler kaliteli bir cihaz malzemesi plastik fakat kadar kaliteli metale dokunuyormuş hissettiriyor isınma sorunu yok maksimum 7980 dereceyi gördüm alır almaz 16gb ram 1tb ssd yükseltmesi yaptım oyunlarda verdiği sonuç beni memnun etti tambir canavar bilgisayar 1tb olsaydi iyi olurdu ürün süper kurulumu zorlasada ürünü gerçekten beğendim fşyata alınabilcek iyi kaliteli laptop 15 senelik bilgisayardan buna geçmenin bir tarifi yoksanki tofaşdan inip ferrariye bindim olduürün sıfır jelatinli olarak işletim sistemi olmadan geliyortam bir oyun canavarıben 50bine aldım paraya alıncak iyi oyuncu laptopunu aldığımı söyliyebilirim ürün başarılı fakat ses çıkışını düşük buldum kaliteli olabilirdi onun dışında gayet kaliteli kendisi canavar ürün gerçekten mükemmel tasarımı performansı ekran kalitesi kasa kalitesi yazılım desteği armoury crate oldukça üst düzey alanlar pişman olmaz yeni nesil frame generation desteklemesinden dolayı rtx 3000 serisi

,Text,Translated Text,Negatif,Notr,Positif,Compound,Sentiment
0,gayet güzel eşime hediye aldım beğendi teşekkü...,I bought a gift to my wife liked my wife liked...,0.03,0.761,0.208,0.9976,Pozitif


In [231]:
df_mainx

,Brand,Model,Price,Favori Number,Rating,Text,Translated Text,Negatif,Notr,Positif,Compound,Sentiment
0,CASPER,G870.1270-EX70R-B i7-12700H 64GB RAM 2TB NVMe ...,65299.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1,LENOVO,"IdeaPad 1 82V700A9TX01 N4020 4GB 256SSD 15.6"" ...",9399.00,1673.0,3.6,güzel görünüyor şarjı 10 saniye felan suruyo a...,"It looks beautiful, the charging is 10 seconds...",0.000,0.862,0.138,0.7155,Pozitif
2,Apple,"Macbook Air M3 16gb 256gb Ssd Macos 13"" Taşına...",44999.00,431.0,5.0,bi sorun olmadı şimdilik şimdi kurulum yapacağ...,"There was no problem for now, now I will set u...",0.000,0.746,0.254,0.6269,Pozitif
3,ASUS,G713QR-K4178 AMD RYZEN 7 5800H 16GB 1TB SSD RT...,81449.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
4,Apple,Apple M4 Max chip with 14?core CPU and 32?core...,157949.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
...,...,...,...,...,...,...,...,...,...,...,...,...
1788,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43399.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1789,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",75716.13,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1790,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",39655.09,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1791,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,24464.30,0.0,5.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif


In [246]:
df_mainx


,Brand,Model,Price,Favori Number,Rating,Text,Translated Text,Negatif,Notr,Positif,Compound,Sentiment
0,CASPER,G870.1270-EX70R-B i7-12700H 64GB RAM 2TB NVMe ...,65299.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1,LENOVO,"IdeaPad 1 82V700A9TX01 N4020 4GB 256SSD 15.6"" ...",9399.00,1673.0,3.6,güzel görünüyor şarjı 10 saniye felan suruyo a...,"It looks beautiful, the charging is 10 seconds...",0.000,0.862,0.138,0.7155,Pozitif
2,Apple,"Macbook Air M3 16gb 256gb Ssd Macos 13"" Taşına...",44999.00,431.0,5.0,bi sorun olmadı şimdilik şimdi kurulum yapacağ...,"There was no problem for now, now I will set u...",0.000,0.746,0.254,0.6269,Pozitif
3,ASUS,G713QR-K4178 AMD RYZEN 7 5800H 16GB 1TB SSD RT...,81449.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
4,Apple,Apple M4 Max chip with 14?core CPU and 32?core...,157949.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
...,...,...,...,...,...,...,...,...,...,...,...,...
1788,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43399.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1789,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",75716.13,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1790,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",39655.09,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1791,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,24464.30,0.0,5.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif


In [230]:
df_mainx.loc[(df_mainx["Text"]=="yorum yoktur")&(df_mainx["Translated Text"].isna()),"Translated Text"]="NO COMMENT"
df_mainx.loc[(df_mainx["Text"]=="yorum yoktur")&(df_mainx["Negatif"].isna()),"Negatif"]=0.688
df_mainx.loc[(df_mainx["Text"]=="yorum yoktur")&(df_mainx["Notr"].isna()),"Notr"]=0.312
df_mainx.loc[(df_mainx["Text"]=="yorum yoktur")&(df_mainx["Positif"].isna()),"Positif"]=0.0
df_mainx.loc[(df_mainx["Text"]=="yorum yoktur")&(df_mainx["Compound"].isna()),"Compound"]=-0.296
df_mainx.loc[(df_mainx["Text"]=="yorum yoktur")&(df_mainx["Sentiment"].isna()),"Sentiment"]="Negatif"


In [242]:
df_mainxx=df_mainx.copy()

In [243]:
for ix in number:
    ix=ix+1000
    result={"Text":[],"Translated Text":[],"Negatif":[],"Notr":[],"Positif":[],"Compound":[],"Sentiment":[]}
    cleaned_texts = [clean_text(yorum) for yorum in df_new["Comments"][ix]]
    tek_metin = " ".join(cleaned_texts)
    sentiment, score = get_sentiment(tek_metin)
    print(f"Metin: {tek_metin}")
    print(f"Çevrilen Metin: {translator.translate(tek_metin, src='tr', dest='en').text}")
    print(f"Polarity Skorları: {score}")
    print(f"Sonuç: {sentiment}")
    result["Text"].append(tek_metin)
    result["Translated Text"].append(translator.translate(tek_metin, src='tr', dest='en').text)
    for key,value in score.items():
        if key=="neg":
            result["Negatif"].append(value)
        elif key=="neu":
            result["Notr"].append(value)
        elif key=="pos":
            result["Positif"].append(value)
        else:
            result["Compound"].append(value)
    result["Sentiment"].append(sentiment)

    res=pd.DataFrame(result)
    ix-=1000
    for col in ["Translated Text","Negatif","Notr","Positif","Compound","Sentiment"]:
        df_mainx[col][ix]=res[col][0]

Metin: paketleme güzel eksiksiz sağlam bir şekilde teslim aldık teşekkür ediyorum beğenerekten kullanıyorum güzel kaliteli indirimdeyken aldım beğenmekten kullanıyorum tek açtırması biraz sıkıntılıydı onu götürüp bilgisayarcıya açtırdık bir seferde yine mahsus on numara bilgisayar fazla araştırdım almadan önce zaten nedenle bir laptop olduğunu biliyordum yüksek performans gerektiren oyunlar falan dışında herseye rahat yapabileceğiniz bir ürün bence ürünüm ertesi gün geldi kargo hızlıydı tşklerimi iletiyorum ayrıca taş paketleme yapmışlar teşekkür ediyorum casper’a laptop güzel oldukça şık hafif yapıda çantada taşınabilecek bir laptop asıl benim beğendim nokta uplift özelliği oldu arayış içerinde olanlara tavsiye ederim modeli mükemmel mükemmel ürün mükemmel mükemmel hızlı güzel rahat kullanılabilen bir bilgisayar herhangi bir sorun çıkmadı gayet güzel ürünün donanımı hızını kalitesini ortaya koyuyor hızlı oyun oynamayan birine göre yeterli pc akıyor bir beklentim olamasına rağmen fazla

C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\User

Metin: beğendimçok iyi bir bilgisayarkargolama iyiydi güzel bilgisayar alınabilir bir öğrencinin gayet kullanabileceği bir ürün oyun oynanabilir tavsiye ederim öğrenci yeterli gayet iyi bir urun beklentinin altında ürün güzl ancak içi bos geliyor onundısında mükemmel yeni ürün aldım maalesef açılmadı iade faturası kesip geri göndereceğim
Çevrilen Metin: I liked a very good computer, a good computer was good, a product that can be used by a product game can be played.
Polarity Skorları: {'neg': 0.0, 'neu': 0.572, 'pos': 0.428, 'compound': 0.9216}
Sonuç: Pozitif


C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\User

Metin: sorunsuz orjinal kutusunda teslim edildi
Çevrilen Metin: It was delivered in the original box without any problems
Polarity Skorları: {'neg': 0.0, 'neu': 0.637, 'pos': 0.363, 'compound': 0.5511}
Sonuç: Pozitif


...

C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\User

Metin: gerek kasa materyalı gerek ekranı gerek donanımıyla üreticileri ideal yönlü taşınabilir bir bilgisayar yanına ganii bir şarj adaptörü almak lazım tuğla adaptör gelmesi taşınabilirliği düşürüyor
Çevrilen Metin: Both Kasa Materyalı and the screen and equipment and manufacturers need to buy a ganii charger adapter next to an ideal portable computer.
Polarity Skorları: {'neg': 0.0, 'neu': 0.866, 'pos': 0.134, 'compound': 0.5267}
Sonuç: Pozitif


C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\User

Metin: eşime kızıma aldım eşim öğretmen kızım üniversite öğrencisi oyun oynayan yok sebepten işinizi fazlasıyla görüyor ben kendi bilgisayarımdan bunu kullanmayı seviyorum hızlı hafif harika bir ürün yaklaşık 1 aydır kullanıyorum hafif yanımda yere tasıyorum sanırım 3500₺ ye almıştım ürün güzel üründen memnun kaldım ürün fiyatına göre iyi kargoda kısa zamanda geldi herkese tavsiye ederim i̇tibar edilmemesi gereken bir satıcı i̇ndirimde aldığım laptopu hemen kargolamadı bir kaç gün sonra arayıp üründe deforme olduğu kargolanmadığını yeni ürün sipariş ettiklerini geldiğinde ancak gönderebileceklerini ilettiler sefer kur farkı olabileceğini söyledi dilerseniz iptal edin başka satıcıdan alın dilerseniz bekleyin dediler ben devamını oku
Çevrilen Metin: I bought my wife to my daughter, my wife, my wife, my daughter, my university student does not play a game for the reason I see your job more than I love to use it from my own computer, I use a wonderful product for about 1 month I use it to 

C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\Users\samet\AppData\Local\Temp\ipykernel_34540\4056013376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mainx[col][ix]=res[col][0]
C:\User

In [42]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samet\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [79]:
from textblob import TextBlob
import nltk 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from wordcloud import WordCloud
import torch 

In [98]:
df_newx=df_new.copy()

In [77]:
df_new = df_new.reset_index(drop=True)

New Comment Data

In [247]:
df_tr= pd.concat([df_main, df_mainx])
df_tr

,Brand,Model,Price,Favori Number,Rating,Text,Translated Text,Negatif,Notr,Positif,Compound,Sentiment
0,CASPER,Nirvana C370.4020-4C00B Intel Celeron N4020 4G...,8750.00,60500.0,4.2,daha çok yeni olduğu için performansına dair y...,I can not comment on the performance of more n...,0.014,0.691,0.295,0.9988,Pozitif
1,LENOVO,NB V15 82YU0123TX RYZEN5-7520U 16GB 512SSD O/B...,13499.00,18500.0,4.5,çok begendim ikincisini sipariş verecem tavsiy...,I would like to order the latter.I have resear...,0.003,0.740,0.257,0.9993,Pozitif
2,Zeiron,Zgx25 Ryzen 5 3600 32GB 3200MHz Ram 512GB SSD ...,18299.00,30300.0,4.6,oğlum için aldım harika bir ürün tavsiye ederi...,I bought a great product for my son I recommen...,0.010,0.686,0.304,0.9996,Pozitif
3,LENOVO,IdeaPad3 Intel Core i3-1215U 8GB 256GB SSD 15....,11005.00,4354.0,3.8,sorunsuz geldi teşekkür ederim satıcıya karne ...,Thank you. I bought it as a report card gift t...,0.047,0.733,0.220,0.9959,Pozitif
4,ASUS,Vivobook 15 Intel Core i5-1235U 8GB 512GB SSD ...,15498.00,7974.0,4.5,kızıma aldım çok memnun 12 yaşındaki ortaokula...,I bought my daughter very satisfied 12 -year -...,0.045,0.651,0.304,0.9992,Pozitif
...,...,...,...,...,...,...,...,...,...,...,...,...
1788,LENOVO,"Ryzen 7 Pro 5850u 48gb 2tb Ssd 14"" W11p 20xls3...",43399.00,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1789,HP,"24GB 4TB SSD RTX4070/8GB 144HZ 16.1"" FHD FDOS ...",75716.13,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1790,HP,"64GB 256GB SSD 15.6"" FHD W11PRO 6S6Z1EA & PER4...",39655.09,0.0,0.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif
1791,LENOVO,V15 G3 Intel Core I7-1255U 24 GB 1 TBSSD FREED...,24464.30,0.0,5.0,yorum yoktur,NO COMMENT,0.688,0.312,0.000,-0.2960,Negatif


In [248]:
df_tr.to_csv('Trendyol_Computer_Comments_NlpClearData.csv', index=False, encoding='utf-8-sig')